In [242]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
def get_info(name):
    info = name.split("__")
    cfg_version = int(info[1].split("=")[1])
    score = float(info[4].split("=")[1].rsplit(".", 1)[0])
    fold_idx = int(info[2].split("=")[1])
    return {
        "cfg_version": cfg_version,
        "score": score,
        "fold_idx": fold_idx,
    }

In [3]:
def merge(cfg_version, score_threshold=0, fold_idx=tuple(range(10000)), scale=False):
    from typing import Iterable
    path = "../predictions_test/"
    predictions = None
    total = 0
    if not isinstance(cfg_version, Iterable):
        cfg_version = [cfg_version]
    for cfgv in cfg_version:
        try:
            for name in os.listdir(path):
                info = get_info(name)
                if (
                    (info["cfg_version"] == cfgv) 
                    and (info["score"] > score_threshold) 
                    and (info["fold_idx"] in fold_idx)
                ):
                    print(f"Processing {name}")
                    a = pd.read_csv(path + name).sort_values(by=["track"])
                    av = np.vstack(a["prediction"].str.split(",").apply(lambda x: list(map(float, x))))
                    if scale:
                        av = StandardScaler().fit_transform(av)
                    # av = np.asarray(a["prediction"].apply(eval).apply(list).tolist())
                    if predictions is None:
                        predictions = av
                    else:
                        predictions += av
                    total += 1
        except Exception as e:
            print(e)
    predictions /= total
    if scale:
        predictions = MinMaxScaler().fit_transform(predictions).clip(min=0 + 1e-8, max=1 - 1e-8)
    predictions_df = pd.DataFrame(
        [
            {"track": track, "prediction": ",".join([str(p) for p in probs])}
            for track, probs in zip(a["track"], predictions)
        ]
    )
    sample_submission = pd.read_csv("../../../VCS/YandexCUP2023/ML/RecSys/input_data/data/sample_submission.csv")
    sample_submission.drop(columns=["prediction"], inplace=True)
    predictions_df = sample_submission.merge(predictions_df, on=["track"])
    n = "_".join(map(str, cfg_version))
    path = f"../submissions/submission__cfg={n}"
    if fold_idx != [-1]:
        path += f"__score_threshold={score_threshold}"
    path += ".csv"
    predictions_df.to_csv(
        path, index=False
    )
    return predictions_df

In [5]:
merge(8, score_threshold=0)

Processing prediction__cfg=8__fold_idx=2__epoch=16__score=0.25270.csv
Processing prediction__cfg=8__fold_idx=1__epoch=17__score=0.24986.csv
Processing prediction__cfg=8__fold_idx=0__epoch=17__score=0.24780.csv
Processing prediction__cfg=8__fold_idx=2__epoch=17__score=0.25395.csv
Processing prediction__cfg=8__fold_idx=1__epoch=16__score=0.24864.csv
Processing prediction__cfg=8__fold_idx=0__epoch=16__score=0.24704.csv
Processing prediction__cfg=8__fold_idx=1__epoch=15__score=0.24533.csv
Processing prediction__cfg=8__fold_idx=2__epoch=15__score=0.25145.csv


,track,prediction
0,17730,"0.073841581625,0.15473362375,0.23961742125,0.2..."
1,32460,"0.210232445,0.46256666375,0.41659273750000003,..."
2,11288,"0.21308714375,0.32346237874999995,0.1953552212..."
3,18523,"0.6206717175,0.106757875,0.2987678975,0.382072..."
4,71342,"0.059608964375,0.020864020625000004,0.04280862..."
...,...,...
25575,40192,"0.28217957125,0.7985012825,0.58858423875,0.095..."
25576,4481,"0.8646494875,0.14652723125,0.06837113825,0.064..."
25577,5257,"0.09504538925,0.068779764375,0.038303698125,0...."
25578,73095,"0.9891893749999999,0.03305615075,0.01965244137..."


In [6]:
merge(9, score_threshold=0)

Processing prediction__cfg=9__fold_idx=2__epoch=19__score=0.25589.csv
Processing prediction__cfg=9__fold_idx=2__epoch=18__score=0.25586.csv
Processing prediction__cfg=9__fold_idx=2__epoch=24__score=0.25588.csv
Processing prediction__cfg=9__fold_idx=1__epoch=20__score=0.25140.csv
Processing prediction__cfg=9__fold_idx=1__epoch=18__score=0.25126.csv
Processing prediction__cfg=9__fold_idx=0__epoch=20__score=0.24885.csv
Processing prediction__cfg=9__fold_idx=2__epoch=23__score=0.25589.csv
Processing prediction__cfg=9__fold_idx=0__epoch=21__score=0.24898.csv
Processing prediction__cfg=9__fold_idx=1__epoch=24__score=0.25122.csv
Processing prediction__cfg=9__fold_idx=0__epoch=17__score=0.24913.csv
Processing prediction__cfg=9__fold_idx=0__epoch=18__score=0.24890.csv
Processing prediction__cfg=9__fold_idx=1__epoch=23__score=0.25117.csv


,track,prediction
0,17730,"0.02436795,0.045193654,0.17961113583333335,0.0..."
1,32460,"0.10043502575,0.551528335,0.43800546083333325,..."
2,11288,"0.05357932199999999,0.32321431500000003,0.0479..."
3,18523,"0.7064180375000001,0.024670806175000004,0.1648..."
4,71342,"0.016589898916666665,0.0029212042083333334,0.0..."
...,...,...
25575,40192,"0.18577336083333332,0.8844984566666666,0.69539..."
25576,4481,"0.9741959324999999,0.06561517433333335,0.01412..."
25577,5257,"0.03130745425,0.019294634333333335,0.006278709..."
25578,73095,"0.998868625,0.004436763775,0.001972133425,0.00..."


In [15]:
merge(14, score_threshold=0)

Processing prediction__cfg=14__fold_idx=1__epoch=15__score=0.25048.csv
Processing prediction__cfg=14__fold_idx=2__epoch=14__score=0.25442.csv
Processing prediction__cfg=14__fold_idx=0__epoch=16__score=0.24908.csv
Processing prediction__cfg=14__fold_idx=2__epoch=15__score=0.25391.csv
Processing prediction__cfg=14__fold_idx=1__epoch=16__score=0.24909.csv
Processing prediction__cfg=14__fold_idx=0__epoch=15__score=0.24960.csv


,track,prediction
0,17730,"0.525777465,0.5341363766666667,0.6311604016666..."
1,32460,"0.60641124,0.7100433433333334,0.71664695166666..."
2,11288,"0.61495069,0.6753237733333334,0.56174374499999..."
3,18523,"0.78768605,0.5160261083333333,0.653018905,0.68..."
4,71342,"0.3832442583333333,0.27101572166666665,0.39415..."
...,...,...
25575,40192,"0.6608994716666667,0.8357663283333333,0.775441..."
25576,4481,"0.9281677616666665,0.551531165,0.5070216783333..."
25577,5257,"0.4876178816666667,0.46307233166666667,0.43115..."
25578,73095,"0.9888416716666666,0.43222799500000003,0.36752..."


In [274]:
def merge_subm(submissions, weights=None, outname="merge_submission.csv", scale=False, rank=False):
    predictions = None
    total = 0
    if weights is None:
        weights = [1] * len(submissions)
    for name, w in zip(submissions, weights):
        print(f"Processing {name}, {w}")
        try:
            a = pd.read_csv(name).sort_values(by=["track"])
            av = np.vstack(a["prediction"].str.split(",").apply(lambda x: list(map(float, x))))
            if scale:
                av = StandardScaler().fit_transform(av)
            elif rank:
                av = av.argsort(axis=0).argsort(axis=0) / len(av)
            if predictions is None:
                predictions = w * av
            else:
                predictions += w * av
            total += w
        except Exception as e:
            print(e)
    if scale:
        predictions = MinMaxScaler().fit_transform(predictions).clip(min=0 + 1e-8, max=1 - 1e-8)
    else:
        predictions /= total
    predictions_df = pd.DataFrame(
        [
            {"track": track, "prediction": ",".join([str(p) for p in probs])}
            for track, probs in zip(a["track"], predictions)
        ]
    )
    sample_submission = pd.read_csv("../../../VCS/YandexCUP2023/ML/RecSys/input_data/data/sample_submission.csv")
    sample_submission.drop(columns=["prediction"], inplace=True)
    predictions_df = sample_submission.merge(predictions_df, on=["track"])
    predictions_df.to_csv(outname, index=False)
    return predictions_df

In [26]:
merge_subm(
    ["../submissions/" + "merge_submission_all.csv"] 
    + ["../submissions/" + "merge_submission_3_0.28280151004107734.csv"],
    outname="../submissions/merge_submission_all_with_best.csv",
    weights=[0.5, 0.5]
)

Processing ../submissions/merge_submission_all.csv
Processing ../submissions/merge_submission_3_0.28280151004107734.csv


,track,prediction
0,17730,"0.06339664331594255,0.10527141905292774,0.1898..."
1,32460,"0.11493191721644869,0.5663593109696101,0.37490..."
2,11288,"0.2289785759680381,0.2266154365534875,0.185718..."
3,18523,"0.4750067253145561,0.07212429572764589,0.28831..."
4,71342,"0.04582814406338291,0.023721621857156597,0.036..."
...,...,...
25575,40192,"0.28149071931380854,0.8501356599965293,0.59310..."
25576,4481,"0.924416258540302,0.15739710139179192,0.055365..."
25577,5257,"0.059129829896640056,0.06398880138391172,0.039..."
25578,73095,"0.9891277378919777,0.016224914933578107,0.0135..."


In [13]:
merge_subm(
    [
        '../submissions/merge_submission_all_with_best_0.28207014233472305.csv',
        '../submissions/merge_submission_2_0.2814506618492104.csv',
        '../submissions/merge_submission_3_0.28280151004107734.csv',
        '../submissions/submission__cfg=9__score_threshold=0__score=0.2695299711324296.csv',
        '../submissions/submission__cfg=7__score_threshold=0__score=0.2720145187109645.csv',
        '../submissions/submission__cfg=8__score_threshold=0__score=0.26542696188270387.csv',
    ],
    outname="../submissions/merge_submission_4.csv",
    weights=[0.25, 0.25, 0.25, 0.15, 0.1, 0.05]
)

Processing ../submissions/merge_submission_all_with_best_0.28207014233472305.csv
Processing ../submissions/merge_submission_2_0.2814506618492104.csv
Processing ../submissions/merge_submission_3_0.28280151004107734.csv
Processing ../submissions/submission__cfg=9__score_threshold=0__score=0.2695299711324296.csv
Processing ../submissions/submission__cfg=7__score_threshold=0__score=0.2720145187109645.csv
Processing ../submissions/submission__cfg=8__score_threshold=0__score=0.26542696188270387.csv


,track,prediction
0,17730,"0.05214073340549268,0.09016715235314267,0.1814..."
1,32460,"0.11067990550007738,0.5649766485916113,0.36679..."
2,11288,"0.19768990125420496,0.2301171037370941,0.15595..."
3,18523,"0.5177320853732136,0.05905211249140782,0.26111..."
4,71342,"0.03670693100046788,0.017595852218639952,0.028..."
...,...,...
25575,40192,"0.26425279629810566,0.8444589400137537,0.62777..."
25576,4481,"0.9368490430749911,0.13883696207069166,0.04557..."
25577,5257,"0.05205382226521309,0.05627619676979763,0.0304..."
25578,73095,"0.991448335230978,0.014377085799169463,0.01030..."


In [18]:
merge_subm(
    [
        "../submissions/merge_submission_4_0.28771512798027166.csv",
        "../submissions/submission__cfg=14__score_threshold=0__score=0.2757725405764561.csv"
    ],
    outname="../submissions/merge_submission_5.csv",
    weights=[0.87, 0.13]
)

Processing ../submissions/merge_submission_4_0.28771512798027166.csv
Processing ../submissions/submission__cfg=14__score_threshold=0__score=0.2757725405764561.csv


,track,prediction
0,17730,"0.11371350851277863,0.1478831515139008,0.23988..."
1,32460,"0.17512497898506735,0.5838353189080352,0.41227..."
2,11288,"0.2519338037911583,0.2879939707846052,0.208705..."
3,18523,"0.5528261007746958,0.11845873195085813,0.31206..."
4,71342,"0.08175678355374039,0.050540435246883425,0.076..."
...,...,...
25575,40192,"0.3158168640960186,0.843328900495299,0.6469735..."
25576,4481,"0.9357204764919089,0.19248720845150175,0.10556..."
25577,5257,"0.10867714998740205,0.10915969430639061,0.0825..."
25578,73095,"0.9911094689676175,0.06869770399527744,0.05674..."


In [21]:
merge_subm(
    [
        "../submissions/merge_submission_4_0.28771512798027166.csv",
        "../submissions/submission__cfg=14__score_threshold=0__score=0.2757725405764561.csv",
         '../predictions_test/prediction__cfg=16__fold_idx=0__epoch=15__score=0.24920.csv',
         '../predictions_test/prediction__cfg=16__fold_idx=1__epoch=15__score=0.25111.csv',
         '../predictions_test/prediction__cfg=16__fold_idx=2__epoch=14__score=0.25452.csv',
         '../predictions_test/prediction__cfg=18__fold_idx=0__epoch=13__score=0.24938.csv',
         '../predictions_test/prediction__cfg=18__fold_idx=1__epoch=14__score=0.24959.csv',
         '../predictions_test/prediction__cfg=18__fold_idx=2__epoch=12__score=0.25364.csv',
         '../predictions_test/prediction__cfg=19__fold_idx=0__epoch=16__score=0.24935.csv',
         '../predictions_test/prediction__cfg=19__fold_idx=1__epoch=15__score=0.24962.csv',
         '../predictions_test/prediction__cfg=19__fold_idx=2__epoch=15__score=0.25543.csv',
         '../predictions_test/prediction__cfg=20__fold_idx=0__epoch=15__score=0.25065.csv',
         '../predictions_test/prediction__cfg=20__fold_idx=1__epoch=15__score=0.25054.csv',
         '../predictions_test/prediction__cfg=20__fold_idx=2__epoch=16__score=0.25388.csv',
         '../predictions_test/prediction__cfg=19__fold_idx=-1__epoch=17__score=-1.csv',
         '../predictions_test/prediction__cfg=20__fold_idx=-1__epoch=19__score=-1.csv',
    ],
    outname="../submissions/merge_submission_6.csv",
    weights=[0.69, 0.05] + [0.02] * 12 + [0.01] * 2
)

Processing ../submissions/merge_submission_4_0.28771512798027166.csv
Processing ../submissions/submission__cfg=14__score_threshold=0__score=0.2757725405764561.csv
Processing ../predictions_test/prediction__cfg=16__fold_idx=0__epoch=15__score=0.24920.csv
Processing ../predictions_test/prediction__cfg=16__fold_idx=1__epoch=15__score=0.25111.csv
Processing ../predictions_test/prediction__cfg=16__fold_idx=2__epoch=14__score=0.25452.csv
Processing ../predictions_test/prediction__cfg=18__fold_idx=0__epoch=13__score=0.24938.csv
Processing ../predictions_test/prediction__cfg=18__fold_idx=1__epoch=14__score=0.24959.csv
Processing ../predictions_test/prediction__cfg=18__fold_idx=2__epoch=12__score=0.25364.csv
Processing ../predictions_test/prediction__cfg=19__fold_idx=0__epoch=16__score=0.24935.csv
Processing ../predictions_test/prediction__cfg=19__fold_idx=1__epoch=15__score=0.24962.csv
Processing ../predictions_test/prediction__cfg=19__fold_idx=2__epoch=15__score=0.25543.csv
Processing ../pred

,track,prediction
0,17730,"0.19744255109978992,0.23305700745700172,0.3244..."
1,32460,"0.26668252359505334,0.6102845818948783,0.47542..."
2,11288,"0.3253103034654014,0.37735780854526146,0.29489..."
3,18523,"0.5950913159075175,0.204875658635738,0.3856003..."
4,71342,"0.15340149990698945,0.10820800551419488,0.1470..."
...,...,...
25575,40192,"0.3884832853290262,0.8439869458261564,0.675187..."
25576,4481,"0.9350982502050769,0.27495737887877714,0.19368..."
25577,5257,"0.18755514924633027,0.1923147138544936,0.15722..."
25578,73095,"0.9903155421927077,0.15124229995142688,0.12806..."


In [23]:
merge(21, score_threshold=0)

Processing prediction__cfg=21__fold_idx=4__epoch=13__score=0.26802.csv
Processing prediction__cfg=21__fold_idx=0__epoch=13__score=0.26361.csv
Processing prediction__cfg=21__fold_idx=1__epoch=26__score=0.26122.csv
Processing prediction__cfg=21__fold_idx=2__epoch=19__score=0.26701.csv
Processing prediction__cfg=21__fold_idx=3__epoch=13__score=0.26030.csv


,track,prediction
0,17730,"0.493898204,0.516043458,0.617555254,0.59522100..."
1,32460,"0.59976608,0.7131351240000001,0.703576062,0.57..."
2,11288,"0.6128468299999998,0.682949462,0.578439468,0.5..."
3,18523,"0.761108314,0.5086968080000001,0.632810508,0.7..."
4,71342,"0.40154949999999995,0.294362758,0.383756574000..."
...,...,...
25575,40192,"0.6624121300000001,0.826079354,0.761584566,0.5..."
25576,4481,"0.9248468599999999,0.58405542,0.502326834,0.45..."
25577,5257,"0.501635564,0.490014008,0.42912370199999994,0...."
25578,73095,"0.987994516,0.437586736,0.37003855799999996,0...."


In [27]:
merge(24, score_threshold=0)

Processing prediction__cfg=24__fold_idx=1__epoch=16__score=0.25392.csv
Processing prediction__cfg=24__fold_idx=2__epoch=13__score=0.25536.csv
Processing prediction__cfg=24__fold_idx=2__epoch=15__score=0.25902.csv
Processing prediction__cfg=24__fold_idx=0__epoch=16__score=0.25308.csv
Processing prediction__cfg=24__fold_idx=0__epoch=15__score=0.25282.csv
Processing prediction__cfg=24__fold_idx=1__epoch=15__score=0.25299.csv


,track,prediction
0,17730,"0.5116802516666666,0.5185673900000001,0.630103..."
1,32460,"0.6229837333333333,0.7031113683333334,0.714979..."
2,11288,"0.6448469533333333,0.6980101900000001,0.586177..."
3,18523,"0.7604863216666665,0.5056525533333334,0.634990..."
4,71342,"0.38059687333333336,0.31826295833333335,0.4078..."
...,...,...
25575,40192,"0.66737964,0.8383055333333332,0.77192366666666..."
25576,4481,"0.9249320416666666,0.5782926899999999,0.511204..."
25577,5257,"0.49106676666666665,0.4767882966666666,0.43078..."
25578,73095,"0.9909543916666667,0.4202318583333333,0.347245..."


In [29]:
merge_subm(
    [
        "../submissions/merge_submission_6__score=0.29027618251524706.csv",
        "../submissions/submission__cfg=24__score_threshold=0__score=0.2771306048143368.csv",
    ],
    outname="../submissions/merge_submission_7.csv",
    weights=[0.9] + [0.1]
)

Processing ../submissions/merge_submission_6__score=0.29027618251524706.csv
Processing ../submissions/submission__cfg=24__score_threshold=0__score=0.2771306048143368.csv


,track,prediction
0,17730,"0.2288663211564776,0.2616080457113016,0.355037..."
1,32460,"0.30231264456888135,0.6195672605387239,0.49938..."
2,11288,"0.3572639684521946,0.40942304669073537,0.32402..."
3,18523,"0.6116308164834324,0.23495334810549753,0.41053..."
4,71342,"0.17612103724962386,0.12921350079610872,0.1730..."
...,...,...
25575,40192,"0.41637292079612365,0.8434188045768741,0.68486..."
25576,4481,"0.934081629351236,0.3052909099908994,0.2254382..."
25577,5257,"0.21790631098836394,0.22076207213571092,0.1845..."
25578,73095,"0.9903794271401035,0.17814125578961754,0.14998..."


In [9]:
merge(26, score_threshold=-2, fold_idx=tuple([-1, *range(10000)]))

Processing prediction__cfg=26__fold_idx=-1__epoch=29__score=-1.csv
Processing prediction__cfg=26__fold_idx=2__epoch=15__score=0.25837.csv
Processing prediction__cfg=26__fold_idx=1__epoch=16__score=0.25443.csv
Processing prediction__cfg=26__fold_idx=2__epoch=14__score=0.25664.csv
Processing prediction__cfg=26__fold_idx=0__epoch=16__score=0.25503.csv
Processing prediction__cfg=26__fold_idx=1__epoch=15__score=0.25278.csv
Processing prediction__cfg=26__fold_idx=0__epoch=15__score=0.25458.csv
Processing prediction__cfg=26__fold_idx=-1__epoch=28__score=-1.csv


,track,prediction
0,17730,"0.518286885,0.53819506,0.63540394125,0.5921274..."
1,32460,"0.5892681775,0.7194760850000002,0.7095151,0.57..."
2,11288,"0.60722322375,0.67377425375,0.5653899812500001..."
3,18523,"0.7785465362499999,0.4791917575,0.627555336249..."
4,71342,"0.38619067625,0.24654028125,0.3858213475,0.692..."
...,...,...
25575,40192,"0.64225192125,0.8434507350000001,0.7698174725,..."
25576,4481,"0.92091114,0.5868281625,0.48916539875,0.454051..."
25577,5257,"0.47359750875,0.49732671625,0.44337430125,0.59..."
25578,73095,"0.99172200125,0.42668571,0.35115768625,0.41780..."


In [24]:
ws = []
xs = []
for x in [f"../submissions/{x}" for x in os.listdir("../submissions/") if "__score" in x]:
    score = float(x.split("__score=")[1].rsplit(".", 1)[0])
    if score < 0.27:
        continue
    xs.append(x)
    ws.append(score)
    
merge_subm(
    xs,
    outname="../submissions/merge_submission_8.csv",
    weights=ws
)

In [28]:
ws = []
xs = []
for x in [f"../submissions/{x}" for x in os.listdir("../submissions/") if "__score" in x]:
    score = float(x.split("__score=")[1].rsplit(".", 1)[0])
    if score < 0.28:
        continue
    xs.append(x)
    ws.append(score)
    
merge_subm(
    xs,
    outname="../submissions/merge_submission_9.csv",
    weights=ws
)

Processing ../submissions/merge_submission_all__score=0.2802971642515184.csv
Processing ../submissions/merge_submission__score=0.2809188604719804.csv
Processing ../submissions/merge_submission_5__score=0.28896974822665383.csv
Processing ../submissions/clf_merge_submission_4__score=0.28498048405976684.csv
Processing ../submissions/merge_submission_6__score=0.29027618251524706.csv
Processing ../submissions/merge_submission_all_with_best__score=0.28207014233472305.csv
Processing ../submissions/merge_submission_4__score=0.28771512798027166.csv
Processing ../submissions/merge_submission_7__score=0.2907024506739986.csv
Processing ../submissions/merge_submission_3__score=0.28280151004107734.csv
Processing ../submissions/merge_submission_2__score=0.2814506618492104.csv


,track,prediction
0,17730,"0.08958185767110337,0.12565702730623002,0.2120..."
1,32460,"0.1460202720746529,0.5758815276234569,0.382180..."
2,11288,"0.23879171446349684,0.25165793188114394,0.1925..."
3,18523,"0.5245781507133066,0.0964494818813097,0.297384..."
4,71342,"0.06565515117338618,0.042707788921550374,0.060..."
...,...,...
25575,40192,"0.2962635490891487,0.8351093621742344,0.632090..."
25576,4481,"0.921223464702339,0.17693484366786372,0.086002..."
25577,5257,"0.08658673097045065,0.09318974073506832,0.0645..."
25578,73095,"0.9725166264701445,0.05114740264251545,0.04248..."


In [34]:
merge(31)

Processing prediction__cfg=31__fold_idx=0__epoch=26__score=0.25486.csv
Processing prediction__cfg=31__fold_idx=1__epoch=16__score=0.25348.csv
Processing prediction__cfg=31__fold_idx=1__epoch=17__score=0.25351.csv
Processing prediction__cfg=31__fold_idx=1__epoch=28__score=0.25363.csv
Processing prediction__cfg=31__fold_idx=0__epoch=22__score=0.25478.csv
Processing prediction__cfg=31__fold_idx=2__epoch=16__score=0.25781.csv
Processing prediction__cfg=31__fold_idx=0__epoch=28__score=0.25498.csv
Processing prediction__cfg=31__fold_idx=2__epoch=17__score=0.25841.csv
Processing prediction__cfg=31__fold_idx=2__epoch=15__score=0.25723.csv


,track,prediction
0,17730,"0.5159731088888889,0.5389116444444445,0.619786..."
1,32460,"0.597966221111111,0.7382951566666667,0.6925989..."
2,11288,"0.6114288077777777,0.7091947277777778,0.610622..."
3,18523,"0.7735694822222222,0.49143704999999993,0.64475..."
4,71342,"0.38464829222222224,0.2716905511111112,0.39783..."
...,...,...
25575,40192,"0.6467506855555556,0.8433208655555555,0.761604..."
25576,4481,"0.9062125833333332,0.6025591344444445,0.534576..."
25577,5257,"0.4707819977777778,0.47482122555555556,0.41121..."
25578,73095,"0.9922088633333335,0.43482788555555557,0.36710..."


In [35]:
merge(35)

Processing prediction__cfg=35__fold_idx=2__epoch=18__score=0.26251.csv
Processing prediction__cfg=35__fold_idx=1__epoch=17__score=0.25597.csv
Processing prediction__cfg=35__fold_idx=0__epoch=17__score=0.25627.csv
Processing prediction__cfg=35__fold_idx=0__epoch=18__score=0.25837.csv
Processing prediction__cfg=35__fold_idx=1__epoch=18__score=0.25709.csv
Processing prediction__cfg=35__fold_idx=2__epoch=17__score=0.26155.csv


,track,prediction
0,17730,"0.47666056500000004,0.50831249,0.58484661,0.56..."
1,32460,"0.56136116,0.674915995,0.6652477316666667,0.52..."
2,11288,"0.5950012566666667,0.6358229350000001,0.538906..."
3,18523,"0.7305331650000001,0.4460468566666667,0.587494..."
4,71342,"0.36464132,0.25764784999999996,0.3526446716666..."
...,...,...
25575,40192,"0.6096095199999999,0.794589305,0.7327738616666..."
25576,4481,"0.8922676316666666,0.53587883,0.49396225333333..."
25577,5257,"0.44595561333333333,0.46000360166666665,0.3908..."
25578,73095,"0.9878376266666667,0.3770076816666667,0.326173..."


In [44]:
merge_subm(
    [
        "../submissions/merge_submission_4__score=0.28771512798027166.csv",
        "../submissions/submission__cfg=14__score_threshold=0__score=0.2757725405764561.csv",
         '../predictions_test/prediction__cfg=16__fold_idx=0__epoch=15__score=0.24920.csv',
         '../predictions_test/prediction__cfg=16__fold_idx=1__epoch=15__score=0.25111.csv',
         '../predictions_test/prediction__cfg=16__fold_idx=2__epoch=14__score=0.25452.csv',
         '../predictions_test/prediction__cfg=18__fold_idx=0__epoch=13__score=0.24938.csv',
         '../predictions_test/prediction__cfg=18__fold_idx=1__epoch=14__score=0.24959.csv',
         '../predictions_test/prediction__cfg=18__fold_idx=2__epoch=12__score=0.25364.csv',
         '../predictions_test/prediction__cfg=19__fold_idx=0__epoch=16__score=0.24935.csv',
         '../predictions_test/prediction__cfg=19__fold_idx=1__epoch=15__score=0.24962.csv',
         '../predictions_test/prediction__cfg=19__fold_idx=2__epoch=15__score=0.25543.csv',
         '../predictions_test/prediction__cfg=20__fold_idx=0__epoch=15__score=0.25065.csv',
         '../predictions_test/prediction__cfg=20__fold_idx=1__epoch=15__score=0.25054.csv',
         '../predictions_test/prediction__cfg=20__fold_idx=2__epoch=16__score=0.25388.csv',
         '../predictions_test/prediction__cfg=19__fold_idx=-1__epoch=17__score=-1.csv',
         '../predictions_test/prediction__cfg=20__fold_idx=-1__epoch=19__score=-1.csv',
         '../submissions/submission__cfg=35__score_threshold=0__score=0.27698060624865495.csv',
         '../submissions/submission__cfg=26__score_threshold=-2__score=0.2762322906091732.csv',
         '../submissions/submission__cfg=24__score_threshold=0__score=0.2771306048143368.csv',
         '../submissions/submission__cfg=21__score_threshold=0__score=0.27559737814616886.csv',
        
    ],
    outname="../submissions/merge_submission_10.csv",
    weights=[0.71, 0.03] + [0.01] * 14 + [0.03] * 4
)

Processing ../submissions/merge_submission_4__score=0.28771512798027166.csv
Processing ../submissions/submission__cfg=14__score_threshold=0__score=0.2757725405764561.csv
Processing ../predictions_test/prediction__cfg=16__fold_idx=0__epoch=15__score=0.24920.csv
Processing ../predictions_test/prediction__cfg=16__fold_idx=1__epoch=15__score=0.25111.csv
Processing ../predictions_test/prediction__cfg=16__fold_idx=2__epoch=14__score=0.25452.csv
Processing ../predictions_test/prediction__cfg=18__fold_idx=0__epoch=13__score=0.24938.csv
Processing ../predictions_test/prediction__cfg=18__fold_idx=1__epoch=14__score=0.24959.csv
Processing ../predictions_test/prediction__cfg=18__fold_idx=2__epoch=12__score=0.25364.csv
Processing ../predictions_test/prediction__cfg=19__fold_idx=0__epoch=16__score=0.24935.csv
Processing ../predictions_test/prediction__cfg=19__fold_idx=1__epoch=15__score=0.24962.csv
Processing ../predictions_test/prediction__cfg=19__fold_idx=2__epoch=15__score=0.25543.csv
Processing 

,track,prediction
0,17730,"0.1861979487378998,0.22055582251073122,0.31226..."
1,32460,"0.25454112823005487,0.6064571000700439,0.46634..."
2,11288,"0.3180871908029855,0.3638597919758368,0.281341..."
3,18523,"0.5892349193224815,0.19046885618389953,0.37268..."
4,71342,"0.1430666438478322,0.09862447385273432,0.13633..."
...,...,...
25575,40192,"0.37844369005915496,0.8418131271797648,0.66952..."
25576,4481,"0.9333905849332433,0.265542375995191,0.1813960..."
25577,5257,"0.17784946054080125,0.1820197012340563,0.14749..."
25578,73095,"0.9906148786314938,0.1379505926474103,0.115752..."


In [64]:
merge_subm([
     '../submissions/merge_submission_all__score=0.2802971642515184.csv',
     '../submissions/merge_submission__score=0.2809188604719804.csv',
     '../submissions/merge_submission_2__score=0.2814506618492104.csv',
     '../submissions/merge_submission_all_with_best__score=0.28207014233472305.csv',
     '../submissions/merge_submission_3__score=0.28280151004107734.csv',
     '../submissions/clf_merge_submission_4__score=0.28498048405976684.csv',
     '../submissions/merge_submission_4__score=0.28771512798027166.csv',
     '../submissions/merge_submission_5__score=0.28896974822665383.csv',
     '../submissions/merge_submission_9__score=0.28985234725624304.csv',
     '../submissions/merge_submission_10__score=0.2900969049214374.csv',
     '../submissions/merge_submission_6__score=0.29027618251524706.csv',
     '../submissions/merge_submission_7__score=0.2907024506739986.csv'
], 
    outname="../submissions/merge_submission_11.csv",
    weights=[0.03494086, 0.06988172, 0.07649278, 0.08092562, 0.0843547 ,
       0.08718963, 0.08962626, 0.09177484, 0.09370415, 0.0954602 ,
       0.09707547, 0.09857377]
)

Processing ../submissions/merge_submission_all__score=0.2802971642515184.csv
Processing ../submissions/merge_submission__score=0.2809188604719804.csv
Processing ../submissions/merge_submission_2__score=0.2814506618492104.csv
Processing ../submissions/merge_submission_all_with_best__score=0.28207014233472305.csv
Processing ../submissions/merge_submission_3__score=0.28280151004107734.csv
Processing ../submissions/clf_merge_submission_4__score=0.28498048405976684.csv
Processing ../submissions/merge_submission_4__score=0.28771512798027166.csv
Processing ../submissions/merge_submission_5__score=0.28896974822665383.csv
Processing ../submissions/merge_submission_9__score=0.28985234725624304.csv
Processing ../submissions/merge_submission_10__score=0.2900969049214374.csv
Processing ../submissions/merge_submission_6__score=0.29027618251524706.csv
Processing ../submissions/merge_submission_7__score=0.2907024506739986.csv


,track,prediction
0,17730,"0.1041145711354739,0.13997732836217774,0.22704..."
1,32460,"0.16273434015593993,0.5799038965104378,0.39404..."
2,11288,"0.2500125494720637,0.26901405863854694,0.20547..."
3,18523,"0.535261888017588,0.11090600905774038,0.307857..."
4,71342,"0.0773481067981095,0.05145479298916414,0.07240..."
...,...,...
25575,40192,"0.30768286707054704,0.8346238105093157,0.63790..."
25576,4481,"0.9215813867600059,0.18976559234021712,0.10062..."
25577,5257,"0.10037249737646699,0.10666066602006874,0.0772..."
25578,73095,"0.9730622109582439,0.06468310255206797,0.05390..."


In [ ]:
merge_subm([
     '../submissions/merge_submission_all__score=0.2802971642515184.csv',
     '../submissions/merge_submission__score=0.2809188604719804.csv',
     '../submissions/merge_submission_2__score=0.2814506618492104.csv',
     '../submissions/merge_submission_all_with_best__score=0.28207014233472305.csv',
     '../submissions/merge_submission_3__score=0.28280151004107734.csv',
     '../submissions/clf_merge_submission_4__score=0.28498048405976684.csv',
     '../submissions/merge_submission_4__score=0.28771512798027166.csv',
     '../submissions/merge_submission_5__score=0.28896974822665383.csv',
     '../submissions/merge_submission_9__score=0.28985234725624304.csv',
     '../submissions/merge_submission_10__score=0.2900969049214374.csv',
     '../submissions/merge_submission_6__score=0.29027618251524706.csv',
     '../submissions/merge_submission_7__score=0.2907024506739986.csv'
], 
    outname="../submissions/merge_submission_11.csv",
    weights=[0.03494086, 0.06988172, 0.07649278, 0.08092562, 0.0843547 ,
       0.08718963, 0.08962626, 0.09177484, 0.09370415, 0.0954602 ,
       0.09707547, 0.09857377]
)

In [65]:
merge(36)

Processing prediction__cfg=36__fold_idx=1__epoch=19__score=0.25960.csv
Processing prediction__cfg=36__fold_idx=2__epoch=17__score=0.26413.csv
Processing prediction__cfg=36__fold_idx=2__epoch=16__score=0.26354.csv
Processing prediction__cfg=36__fold_idx=0__epoch=18__score=0.26173.csv
Processing prediction__cfg=36__fold_idx=0__epoch=17__score=0.26098.csv
Processing prediction__cfg=36__fold_idx=1__epoch=17__score=0.25950.csv


,track,prediction
0,17730,"0.468513735,0.519304635,0.5622960433333334,0.5..."
1,32460,"0.5650396366666667,0.6718854533333333,0.662459..."
2,11288,"0.5941465616666667,0.6410248749999999,0.576197..."
3,18523,"0.7253101966666667,0.4755504733333333,0.595363..."
4,71342,"0.38352608833333335,0.30409677500000004,0.3795..."
...,...,...
25575,40192,"0.6122982783333333,0.804070095,0.73027736,0.45..."
25576,4481,"0.8943314133333334,0.5448268683333334,0.486200..."
25577,5257,"0.4646999,0.46454271500000005,0.41168734,0.550..."
25578,73095,"0.9840403766666667,0.4017314616666667,0.339810..."


In [66]:
merge(37)

Processing prediction__cfg=37__fold_idx=1__epoch=15__score=0.26000.csv
Processing prediction__cfg=37__fold_idx=0__epoch=14__score=0.26223.csv
Processing prediction__cfg=37__fold_idx=2__epoch=14__score=0.26375.csv


,track,prediction
0,17730,"0.5172907000000001,0.5348419333333333,0.635928..."
1,32460,"0.6028550833333334,0.7246884800000001,0.697824..."
2,11288,"0.60848165,0.7049917166666666,0.58834711666666..."
3,18523,"0.76699021,0.5443199366666667,0.65184386666666..."
4,71342,"0.43531063333333336,0.29360175,0.4140804533333..."
...,...,...
25575,40192,"0.6552916333333333,0.8321303933333333,0.764237..."
25576,4481,"0.9212373133333335,0.5841242166666666,0.505555..."
25577,5257,"0.5277850466666666,0.5119393766666667,0.451846..."
25578,73095,"0.98775938,0.46339194666666667,0.3596494966666..."


In [67]:
merge(41)

Processing prediction__cfg=41__fold_idx=2__epoch=18__score=0.26134.csv
Processing prediction__cfg=41__fold_idx=1__epoch=18__score=0.25818.csv
Processing prediction__cfg=41__fold_idx=0__epoch=18__score=0.25792.csv
Processing prediction__cfg=41__fold_idx=2__epoch=16__score=0.26070.csv
Processing prediction__cfg=41__fold_idx=1__epoch=19__score=0.25893.csv
Processing prediction__cfg=41__fold_idx=0__epoch=17__score=0.25652.csv


,track,prediction
0,17730,"0.480142635,0.5081560816666667,0.5775825266666..."
1,32460,"0.5595212483333333,0.691897345,0.6710617266666..."
2,11288,"0.566002605,0.6646319683333334,0.56608632,0.48..."
3,18523,"0.7224686216666667,0.47871697333333335,0.60821..."
4,71342,"0.37945075,0.2927692,0.36045942833333333,0.715..."
...,...,...
25575,40192,"0.61329654,0.8058789083333334,0.72403193333333..."
25576,4481,"0.8890990066666666,0.5517753599999999,0.464958..."
25577,5257,"0.4261615633333333,0.4368934633333334,0.394296..."
25578,73095,"0.9890413916666666,0.3766118716666667,0.314194..."


In [117]:
merge([43, 44, 45, 46], fold_idx=range(-100, 100), score_threshold=-2)

Processing prediction__cfg=43__fold_idx=1__epoch=17__score=0.25667.csv
Processing prediction__cfg=43__fold_idx=-1__epoch=18__score=-1.csv
Processing prediction__cfg=43__fold_idx=1__epoch=16__score=0.25620.csv
Processing prediction__cfg=43__fold_idx=2__epoch=16__score=0.25976.csv
Processing prediction__cfg=43__fold_idx=0__epoch=16__score=0.25611.csv
Processing prediction__cfg=43__fold_idx=0__epoch=17__score=0.25746.csv
Processing prediction__cfg=43__fold_idx=-1__epoch=19__score=-1.csv
Processing prediction__cfg=43__fold_idx=2__epoch=18__score=0.26178.csv
Processing prediction__cfg=44__fold_idx=0__epoch=18__score=0.25767.csv
Processing prediction__cfg=44__fold_idx=2__epoch=18__score=0.25624.csv
Processing prediction__cfg=44__fold_idx=1__epoch=17__score=0.25373.csv
Processing prediction__cfg=44__fold_idx=0__epoch=17__score=0.25425.csv
Processing prediction__cfg=44__fold_idx=-1__epoch=18__score=-1.csv
Processing prediction__cfg=44__fold_idx=-1__epoch=19__score=-1.csv
Processing prediction_

,track,prediction
0,17730,"0.4617941171874999,0.4989631878125,0.571129558..."
1,32460,"0.55365263625,0.6776293846875,0.65672514562500..."
2,11288,"0.5715225759374999,0.6223368015624999,0.563100..."
3,18523,"0.7170499978124999,0.46058181906250006,0.60508..."
4,71342,"0.35570052593750007,0.2698137024999999,0.37879..."
...,...,...
25575,40192,"0.6035011853125,0.8039391321875001,0.731815013..."
25576,4481,"0.883579619375,0.5303862637500001,0.48406685,0..."
25577,5257,"0.44406989687500004,0.47173465937500003,0.3964..."
25578,73095,"0.9881476712499999,0.3738721134375,0.319415017..."


In [ ]:
merge_subm([
    os.path.join("../predictions_test", x) for x in os.listdir("../predictions_test")
], 
    outname="../submissions/merge_submission_12.csv",
)

In [60]:
merge_subm([
    os.path.join("../predictions_test", x) for x in os.listdir("../predictions_test") 
    if get_info(x)["score"] > 0.248
], 
    outname="../submissions/merge_submission_13.csv",
)

Processing ../predictions_test/prediction__cfg=44__fold_idx=0__epoch=18__score=0.25767.csv
Processing ../predictions_test/prediction__cfg=42__fold_idx=1__epoch=17__score=0.25192.csv
Processing ../predictions_test/prediction__cfg=44__fold_idx=2__epoch=18__score=0.25624.csv
Processing ../predictions_test/prediction__cfg=37__fold_idx=1__epoch=15__score=0.26000.csv
Processing ../predictions_test/prediction__cfg=44__fold_idx=1__epoch=17__score=0.25373.csv
Processing ../predictions_test/prediction__cfg=19__fold_idx=2__epoch=15__score=0.25543.csv
Processing ../predictions_test/prediction__cfg=38__fold_idx=2__epoch=14__score=0.26096.csv
Processing ../predictions_test/prediction__cfg=24__fold_idx=1__epoch=16__score=0.25392.csv
Processing ../predictions_test/prediction__cfg=46__fold_idx=0__epoch=16__score=0.25581.csv
Processing ../predictions_test/prediction__cfg=8__fold_idx=2__epoch=16__score=0.25270.csv
Processing ../predictions_test/prediction__cfg=43__fold_idx=1__epoch=17__score=0.25667.csv


Processing ../predictions_test/prediction__cfg=8__fold_idx=2__epoch=15__score=0.25145.csv
Processing ../predictions_test/prediction__cfg=9__fold_idx=2__epoch=23__score=0.25589.csv
Processing ../predictions_test/prediction__cfg=9__fold_idx=0__epoch=21__score=0.24898.csv
Processing ../predictions_test/prediction__cfg=47__fold_idx=0__epoch=16__score=0.25637.csv
Processing ../predictions_test/prediction__cfg=45__fold_idx=1__epoch=16__score=0.25425.csv
Processing ../predictions_test/prediction__cfg=9__fold_idx=1__epoch=24__score=0.25122.csv
Processing ../predictions_test/prediction__cfg=26__fold_idx=0__epoch=16__score=0.25503.csv
Processing ../predictions_test/prediction__cfg=24__fold_idx=0__epoch=15__score=0.25282.csv
Processing ../predictions_test/prediction__cfg=46__fold_idx=1__epoch=16__score=0.25720.csv
Processing ../predictions_test/prediction__cfg=16__fold_idx=0__epoch=15__score=0.24920.csv
Processing ../predictions_test/prediction__cfg=22__fold_idx=2__epoch=15__score=0.25487.csv
Pro

,track,prediction
0,17730,"0.4450550954312497,0.47311678161250015,0.55847..."
1,32460,"0.5344222953687501,0.6800154515624998,0.660019..."
2,11288,"0.5413876214000001,0.6289551914375004,0.522401..."
3,18523,"0.7396774313124999,0.4424062659631251,0.583129..."
4,71342,"0.3452050340375,0.254407892040625,0.3477695199..."
...,...,...
25575,40192,"0.5864107584374998,0.8262036049999997,0.740511..."
25576,4481,"0.9104267355625003,0.5062747307624997,0.440897..."
25577,5257,"0.4228871011124998,0.42304738046874996,0.37156..."
25578,73095,"0.9885973243125005,0.36540607873312486,0.31081..."


In [65]:
merge_subm([
    "../submissions/merge_submission_7__score=0.2907024506739986.csv",
    "../submissions/submission__cfg=43_44_45_46__score_threshold=-2__score=0.28227115176735484.csv",
],weights=[0.9, 0.1], outname="../submissions/merge_submission_14")

Processing ../submissions/merge_submission_7__score=0.2907024506739986.csv
Processing ../submissions/submission__cfg=43_44_45_46__score_threshold=-2__score=0.28227115176735484.csv


,track,prediction
0,17730,"0.2521591007595798,0.28534355992142146,0.37664..."
1,32460,"0.3274466437369932,0.6253734729536016,0.515116..."
2,11288,"0.37868982920072514,0.4307144221779119,0.34792..."
3,18523,"0.6221727346163392,0.2575161952011978,0.429993..."
4,71342,"0.19407898611841148,0.14327352096649784,0.1936..."
...,...,...
25575,40192,"0.4350857472477613,0.8394708373379367,0.689556..."
25576,4481,"0.9290314283536124,0.32780044536680947,0.25130..."
25577,5257,"0.24052266957702756,0.24585933085963985,0.2057..."
25578,73095,"0.9901562515510932,0.1977143415544058,0.166928..."


In [28]:
merge_subm([
    "../submissions/merge_submission_4__score=0.28771512798027166.csv",
    "../submissions/merge_submission_14__score=0.2912586993297299.csv",
],weights=[0.5, 0.5], outname="../submissions/merge_submission_15.csv")

Processing ../submissions/merge_submission_4__score=0.28771512798027166.csv
Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv


,track,prediction
0,17730,"0.15214991708253625,0.18775535613728206,0.2790..."
1,32460,"0.2190632746185353,0.5951750607726065,0.440957..."
2,11288,"0.28818986522746504,0.33041576295750297,0.2519..."
3,18523,"0.5699524099947764,0.1582841538463028,0.345553..."
4,71342,"0.11539295855943968,0.0804346865925689,0.11106..."
...,...,...
25575,40192,"0.3496692717729335,0.8419648886758452,0.658666..."
25576,4481,"0.9329402357143017,0.23331870371875058,0.14843..."
25577,5257,"0.14628824592112033,0.15106776381471873,0.1181..."
25578,73095,"0.9908022933910356,0.10604571367678763,0.08861..."


In [52]:
merge_subm([
    os.path.join("../predictions_test", x) for x in os.listdir("../predictions_test")
], 
    outname="../submissions/merge_submission_16.csv",
    scale=True
)

Processing ../predictions_test/prediction__cfg=58__fold_idx=0__epoch=15__score=0.25944.csv
Processing ../predictions_test/prediction__cfg=44__fold_idx=0__epoch=18__score=0.25767.csv
Processing ../predictions_test/prediction__cfg=42__fold_idx=1__epoch=17__score=0.25192.csv
Processing ../predictions_test/prediction__cfg=5__fold_idx=2__epoch=17__score=0.24446.csv
Processing ../predictions_test/prediction__cfg=44__fold_idx=2__epoch=18__score=0.25624.csv
Processing ../predictions_test/prediction__cfg=38__fold_idx=-1__epoch=18__score=-1.csv
Processing ../predictions_test/prediction__cfg=37__fold_idx=1__epoch=15__score=0.26000.csv
Processing ../predictions_test/prediction__cfg=48__fold_idx=0__epoch=13__score=0.24586.csv
Processing ../predictions_test/prediction__cfg=44__fold_idx=1__epoch=17__score=0.25373.csv
Processing ../predictions_test/prediction__cfg=48__fold_idx=2__epoch=12__score=0.24541.csv
Processing ../predictions_test/prediction__cfg=19__fold_idx=2__epoch=15__score=0.25543.csv
Proc

Processing ../predictions_test/prediction__cfg=7__fold_idx=1__epoch=10__score=0.23714.csv
Processing ../predictions_test/prediction__cfg=43__fold_idx=1__epoch=16__score=0.25620.csv
Processing ../predictions_test/prediction__cfg=41__fold_idx=-1__epoch=19__score=-1.csv
Processing ../predictions_test/prediction__cfg=43__fold_idx=2__epoch=16__score=0.25976.csv
Processing ../predictions_test/prediction__cfg=30__fold_idx=-1__epoch=23__score=-1.csv
Processing ../predictions_test/prediction__cfg=46__fold_idx=-1__epoch=19__score=-1.csv
Processing ../predictions_test/prediction__cfg=41__fold_idx=2__epoch=18__score=0.26134.csv
Processing ../predictions_test/prediction__cfg=49__fold_idx=2__epoch=13__score=0.24769.csv
Processing ../predictions_test/prediction__cfg=20__fold_idx=1__epoch=15__score=0.25054.csv
Processing ../predictions_test/prediction__cfg=7__fold_idx=1__epoch=12__score=0.23640.csv
Processing ../predictions_test/prediction__cfg=39__fold_idx=0__epoch=13__score=0.25492.csv
Processing ..

Processing ../predictions_test/prediction__cfg=31__fold_idx=2__epoch=16__score=0.25781.csv
Processing ../predictions_test/prediction__cfg=53__fold_idx=-2__epoch=1__score=-2.csv
Processing ../predictions_test/prediction__cfg=46__fold_idx=2__epoch=18__score=0.25646.csv
Processing ../predictions_test/prediction__cfg=8__fold_idx=-1__epoch=11__score=-1.csv
Processing ../predictions_test/prediction__cfg=7__fold_idx=0__epoch=11__score=0.23508.csv
Processing ../predictions_test/prediction__cfg=48__fold_idx=1__epoch=13__score=0.24866.csv
Processing ../predictions_test/prediction__cfg=4__fold_idx=1__epoch=11__score=0.22735.csv
Processing ../predictions_test/prediction__cfg=46__fold_idx=-1__epoch=18__score=-1.csv
Processing ../predictions_test/prediction__cfg=30__fold_idx=-1__epoch=22__score=-1.csv
Processing ../predictions_test/prediction__cfg=8__fold_idx=2__epoch=15__score=0.25145.csv
Processing ../predictions_test/prediction__cfg=9__fold_idx=2__epoch=23__score=0.25589.csv
Processing ../predict

Processing ../predictions_test/prediction__cfg=7__fold_idx=-1__epoch=10__score=-1.csv
Processing ../predictions_test/prediction__cfg=47__fold_idx=0__epoch=15__score=0.25241.csv
Processing ../predictions_test/prediction__cfg=30__fold_idx=1__epoch=16__score=0.25377.csv
Processing ../predictions_test/prediction__cfg=18__fold_idx=-1__epoch=19__score=-1.csv
Processing ../predictions_test/prediction__cfg=39__fold_idx=0__epoch=14__score=0.25720.csv
Processing ../predictions_test/prediction__cfg=41__fold_idx=0__epoch=17__score=0.25652.csv
Processing ../predictions_test/prediction__cfg=29__fold_idx=1__epoch=16__score=0.25337.csv
Processing ../predictions_test/prediction__cfg=46__fold_idx=2__epoch=19__score=0.25772.csv
Processing ../predictions_test/prediction__cfg=44__fold_idx=1__epoch=18__score=0.25613.csv
Processing ../predictions_test/prediction__cfg=5__fold_idx=2__epoch=16__score=0.24452.csv
Processing ../predictions_test/prediction__cfg=47__fold_idx=1__epoch=18__score=0.25474.csv
Processin

,track,prediction
0,17730,"0.0009403227719178115,0.0009597154074433449,0...."
1,32460,"0.001264177016968287,0.001855074882127399,0.00..."
2,11288,"0.0013429604931053112,0.001533046949324226,0.0..."
3,18523,"0.0020691765979810087,0.0008352463793661624,0...."
4,71342,"0.0005911086371396377,9.879929067414482e-05,0...."
...,...,...
25575,40192,"0.001521880184676998,0.0025235633687536335,0.0..."
25576,4481,"0.00287142643212514,0.0010843476400447596,0.00..."
25577,5257,"0.0008497043866728571,0.0007314378689672313,0...."
25578,73095,"0.0031690769439458866,0.0004925940872482793,0...."


In [64]:
merge_subm([
    os.path.join("../predictions_test", x) for x in os.listdir("../predictions_test") 
    if get_info(x)["score"] > 0.26
], 
    outname="../submissions/merge_submission_17.csv",
    scale=True,
)

Processing ../predictions_test/prediction__cfg=38__fold_idx=2__epoch=14__score=0.26096.csv
Processing ../predictions_test/prediction__cfg=58__fold_idx=1__epoch=12__score=0.26408.csv
Processing ../predictions_test/prediction__cfg=21__fold_idx=4__epoch=13__score=0.26802.csv
Processing ../predictions_test/prediction__cfg=36__fold_idx=2__epoch=17__score=0.26413.csv
Processing ../predictions_test/prediction__cfg=41__fold_idx=2__epoch=18__score=0.26134.csv
Processing ../predictions_test/prediction__cfg=35__fold_idx=2__epoch=18__score=0.26251.csv
Processing ../predictions_test/prediction__cfg=21__fold_idx=0__epoch=13__score=0.26361.csv
Processing ../predictions_test/prediction__cfg=36__fold_idx=2__epoch=16__score=0.26354.csv
Processing ../predictions_test/prediction__cfg=21__fold_idx=1__epoch=26__score=0.26122.csv
Processing ../predictions_test/prediction__cfg=21__fold_idx=2__epoch=19__score=0.26701.csv
Processing ../predictions_test/prediction__cfg=36__fold_idx=0__epoch=18__score=0.26173.csv

,track,prediction
0,17730,"0.35230080681626297,0.3528575422283401,0.56052..."
1,32460,"0.4847834850015128,0.5724920707769342,0.724362..."
2,11288,"0.4995848859210531,0.5406751793330155,0.522311..."
3,18523,"0.6814360784050175,0.29597355733420266,0.60449..."
4,71342,"0.2629290525326455,0.052421086046370746,0.2518..."
...,...,...
25575,40192,"0.5522146889761482,0.7634719805477155,0.822273..."
25576,4481,"0.880817130452008,0.40925783296640805,0.416657..."
25577,5257,"0.3364459966607569,0.2776647560712965,0.271439..."
25578,73095,"0.9896222951087512,0.22148532966693052,0.17628..."


In [67]:
merge_subm([
    "../submissions/merge_submission_4__score=0.28771512798027166.csv",
    "../submissions/merge_submission_14__score=0.2912586993297299.csv",
],weights=[0.5, 0.5], outname="../submissions/merge_submission_15.csv", scale=True)

Processing ../submissions/merge_submission_4__score=0.28771512798027166.csv
Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv


,track,prediction
0,17730,"0.1124589545536126,0.13853323432664782,0.27349..."
1,32460,"0.1848808733191537,0.570940952220341,0.4664855..."
2,11288,"0.25630226611165285,0.2935512753648658,0.24056..."
3,18523,"0.5518484672820223,0.1068113928372516,0.352368..."
4,71342,"0.07053576849472276,0.01743008518186845,0.0687..."
...,...,...
25575,40192,"0.3212075793991142,0.833981718531514,0.7246097..."
25576,4481,"0.9317213175282297,0.18749864377560413,0.11626..."
25577,5257,"0.10546954649787965,0.09837971278127772,0.0778..."
25578,73095,"0.9938376013008071,0.04914674593307433,0.04113..."


In [6]:
merge([58, 59], fold_idx=range(-50, 1000), score_threshold=-2)

Processing prediction__cfg=58__fold_idx=0__epoch=15__score=0.25944.csv
Processing prediction__cfg=58__fold_idx=3__epoch=12__score=0.26978.csv
Processing prediction__cfg=58__fold_idx=1__epoch=12__score=0.26408.csv
Processing prediction__cfg=58__fold_idx=-1__epoch=18__score=-1.csv
Processing prediction__cfg=58__fold_idx=3__epoch=13__score=0.27106.csv
Processing prediction__cfg=58__fold_idx=0__epoch=16__score=0.25950.csv
Processing prediction__cfg=58__fold_idx=4__epoch=12__score=0.26145.csv
Processing prediction__cfg=58__fold_idx=2__epoch=14__score=0.26187.csv
Processing prediction__cfg=58__fold_idx=2__epoch=16__score=0.26196.csv
Processing prediction__cfg=58__fold_idx=0__epoch=18__score=0.25964.csv
Processing prediction__cfg=58__fold_idx=1__epoch=13__score=0.26430.csv
Processing prediction__cfg=58__fold_idx=4__epoch=13__score=0.26331.csv
Processing prediction__cfg=58__fold_idx=-1__epoch=19__score=-1.csv
Processing prediction__cfg=59__fold_idx=-1__epoch=20__score=-1.csv


,track,prediction
0,17730,"0.1971809939285714,0.24980042357142862,0.36088..."
1,32460,"0.34234086214285714,0.5403612799999999,0.55517..."
2,11288,"0.3668134599999999,0.4303303035714286,0.305167..."
3,18523,"0.6241567671428572,0.21101865435714282,0.39553..."
4,71342,"0.11774625014285714,0.07180918016428572,0.1199..."
...,...,...
25575,40192,"0.4038114885714285,0.7796578278571429,0.645219..."
25576,4481,"0.8827733621428573,0.31684729142857143,0.20579..."
25577,5257,"0.20755504071428568,0.18774899499999997,0.1222..."
25578,73095,"0.9876310914285715,0.1319152492714286,0.089233..."


In [11]:
merge_subm([
    "../submissions/merge_submission_4__score=0.28771512798027166.csv",
    "../submissions/merge_submission_14__score=0.2912586993297299.csv",
    "../submissions/merge_submission_15__score=0.289886916346524.csv",
],weights=[0.45, 0.45, 0.1], outname="../submissions/merge_submission_18.csv", scale=True)

Processing ../submissions/merge_submission_4__score=0.28771512798027166.csv
Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv
Processing ../submissions/merge_submission_15__score=0.289886916346524.csv


,track,prediction
0,17730,"0.11169225501297053,0.13772638762392947,0.2727..."
1,32460,"0.18396727972994237,0.5707876174529151,0.46589..."
2,11288,"0.2555741299186742,0.29261549712281254,0.23987..."
3,18523,"0.5514363789624552,0.10600121139496393,0.35173..."
4,71342,"0.07006228286788052,0.017165069660828536,0.068..."
...,...,...
25575,40192,"0.3205068604011289,0.8341137040730919,0.724334..."
25576,4481,"0.9318029856865991,0.18670463684925503,0.11558..."
25577,5257,"0.1047797799053749,0.0976345761365322,0.077411..."
25578,73095,"0.9938428076421587,0.04847313578918072,0.04079..."


In [43]:
merge_subm([
    "../submissions/merge_submission_4__score=0.28771512798027166.csv",
    "../submissions/merge_submission_14__score=0.2912586993297299.csv",
    "../submissions/submission__cfg=58_59__score_threshold=-2.csv",
],weights=[0.5, 0.4, 0.1], outname="../submissions/merge_submission_16.csv", scale=True)

Processing ../submissions/merge_submission_4__score=0.28771512798027166.csv
Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv
Processing ../submissions/submission__cfg=58_59__score_threshold=-2.csv


,track,prediction
0,17730,"0.11213330305474944,0.13696888897199244,0.2782..."
1,32460,"0.19211606510535603,0.5623386410284663,0.47839..."
2,11288,"0.26001274973020466,0.2955971826307713,0.24212..."
3,18523,"0.5552735049396947,0.10405681369675118,0.35513..."
4,71342,"0.06808973076595465,0.012171881842250987,0.066..."
...,...,...
25575,40192,"0.32225131877882385,0.8275841861070141,0.72710..."
25576,4481,"0.9271645837083982,0.18867904022471316,0.11733..."
25577,5257,"0.10775280859294395,0.09430154381752875,0.0745..."
25578,73095,"0.9939053732537384,0.044340419678651766,0.0386..."


In [59]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv",
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
],weights=[0.65, 0.35], outname="../submissions/merge_submission_17.csv", scale=True)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv


,track,prediction
0,17730,"0.13812577310689012,0.16437003187797633,0.2576..."
1,32460,"0.20836950376473742,0.5373098383206845,0.50635..."
2,11288,"0.26627070803837893,0.3063989032686877,0.23961..."
3,18523,"0.5585402633779627,0.1254162342003633,0.375985..."
4,71342,"0.07931006890824277,0.023728322826548076,0.076..."
...,...,...
25575,40192,"0.31346505713924383,0.8307739973402576,0.74134..."
25576,4481,"0.9439847988394512,0.18689612098458844,0.12759..."
25577,5257,"0.11649391347412758,0.10653935749195231,0.0859..."
25578,73095,"0.9954257102774169,0.06706155083692433,0.05041..."


In [63]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv",
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
],weights=[0.8, 0.2], outname="../submissions/merge_submission_18.csv", scale=True)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv


,track,prediction
0,17730,"0.15303309609335147,0.1790673355893644,0.29644..."
1,32460,"0.2292638437403412,0.5571956736396947,0.515189..."
2,11288,"0.286781183052619,0.33103640930232436,0.270124..."
3,18523,"0.56964845013691,0.14342895945898992,0.3935091..."
4,71342,"0.09164306413893009,0.029526169030415494,0.091..."
...,...,...
25575,40192,"0.3408566327864377,0.8279179928396941,0.746250..."
25576,4481,"0.9356872976171887,0.21290283585858066,0.15245..."
25577,5257,"0.1352369415226355,0.1270758776583612,0.103349..."
25578,73095,"0.9944585051261137,0.08095303971666662,0.06144..."


In [67]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv",
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
],weights=[0.9, 0.1], outname="../submissions/merge_submission_19.csv", scale=True)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv


,track,prediction
0,17730,"0.1625894101354211,0.18844641750500365,0.32207..."
1,32460,"0.24265789313280467,0.5698858161366068,0.52103..."
2,11288,"0.2999281923506649,0.34675882967440164,0.29030..."
3,18523,"0.5767621869462335,0.15492377708282795,0.40510..."
4,71342,"0.09954965822881662,0.03322606361980296,0.1009..."
...,...,...
25575,40192,"0.3584150197762339,0.8260954340433171,0.749497..."
25576,4481,"0.9303521838607296,0.22949901662409403,0.16889..."
25577,5257,"0.14725302464990014,0.14018125458227143,0.1148..."
25578,73095,"0.9938227237075155,0.08981789103169663,0.06873..."


In [75]:
merge([60, 61], fold_idx=range(-50, 1000), score_threshold=-2, scale=True)

Processing prediction__cfg=60__fold_idx=4__epoch=12__score=0.25938.csv
Processing prediction__cfg=60__fold_idx=2__epoch=17__score=0.26002.csv
Processing prediction__cfg=60__fold_idx=0__epoch=18__score=0.26564.csv
Processing prediction__cfg=60__fold_idx=2__epoch=15__score=0.25994.csv
Processing prediction__cfg=60__fold_idx=3__epoch=14__score=0.26526.csv
Processing prediction__cfg=60__fold_idx=3__epoch=17__score=0.26536.csv
Processing prediction__cfg=60__fold_idx=-1__epoch=19__score=-1.csv
Processing prediction__cfg=60__fold_idx=-1__epoch=18__score=-1.csv
Processing prediction__cfg=60__fold_idx=4__epoch=13__score=0.26138.csv
Processing prediction__cfg=60__fold_idx=1__epoch=13__score=0.26613.csv
Processing prediction__cfg=60__fold_idx=1__epoch=12__score=0.26352.csv
Processing prediction__cfg=60__fold_idx=0__epoch=19__score=0.26567.csv
Processing prediction__cfg=61__fold_idx=2__epoch=12__score=0.25703.csv
Processing prediction__cfg=61__fold_idx=0__epoch=11__score=0.25655.csv
Processing pre

,track,prediction
0,17730,"0.24156976731443652,0.2781026579573327,0.45276..."
1,32460,"0.3804668785832948,0.5669623796974818,0.654837..."
2,11288,"0.4060813206028633,0.4941821558469041,0.451326..."
3,18523,"0.6250322523959704,0.23765994367079898,0.53992..."
4,71342,"0.12815381321083547,0.01582982399882482,0.1512..."
...,...,...
25575,40192,"0.4569417427890928,0.7635440450860655,0.808786..."
25576,4481,"0.8844745845755803,0.35251932852443457,0.33014..."
25577,5257,"0.23597801909555788,0.20944709176731952,0.1626..."
25578,73095,"0.989246104885547,0.15674610887705626,0.129124..."


In [90]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv",
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/submission__cfg=60_61__score_threshold=-2.csv"
],weights=[0.8, 0.1, 0.1], outname="../submissions/merge_submission_20.csv", scale=True)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv
Processing ../submissions/submission__cfg=60_61__score_threshold=-2.csv


,track,prediction
0,17730,"0.1700280077866026,0.19357212893275882,0.33160..."
1,32460,"0.2564622526431287,0.5662193990200357,0.533221..."
2,11288,"0.310232748157373,0.3585665747148472,0.3032223..."
3,18523,"0.5813863131926054,0.15889406144817203,0.41659..."
4,71342,"0.10138350809141694,0.026078602127321715,0.103..."
...,...,...
25575,40192,"0.36748702588500604,0.8184456449703497,0.75524..."
25576,4481,"0.9258946839303125,0.23788049132280104,0.18198..."
25577,5257,"0.1555393560064724,0.14240448899004038,0.11749..."
25578,73095,"0.993646654550139,0.09193608413462007,0.072920..."


In [96]:
merge_subm([
    "../submissions/mlp_merge_submission_20__score=0.2952258507364594.csv",
    "../submissions/merge_submission_20__score=0.2952258507364594.csv",
],weights=[0.1, 0.9], outname="../submissions/merge_submission_21.csv", scale=True)

Processing ../submissions/mlp_merge_submission_20__score=0.2952258507364594.csv
Processing ../submissions/merge_submission_20__score=0.2952258507364594.csv


,track,prediction
0,17730,"0.15349159416902708,0.17573589694398012,0.3783..."
1,32460,"0.27986119733568376,0.5752558598226574,0.52117..."
2,11288,"0.32996873845031055,0.39304172016899197,0.2910..."
3,18523,"0.5332221077519731,0.1839256138915128,0.396006..."
4,71342,"0.1516153261426721,0.047155337598628,0.1005584..."
...,...,...
25575,40192,"0.3626109359379436,0.8253530295271727,0.716924..."
25576,4481,"0.8857432765607212,0.28682301195519533,0.18023..."
25577,5257,"0.19119903978152963,0.13866335611281444,0.1127..."
25578,73095,"0.9112302592276667,0.1601100656777384,0.068952..."


In [101]:
merge_subm([
    "../submissions/mlp_merge_submission_20__score=0.2952258507364594.csv",
    "../submissions/logreg_merge_submission_20__score=0.2952258507364594.csv",
],weights=[0.5, 0.5], outname="../submissions/merge_submission_22.csv", scale=True)

Processing ../submissions/mlp_merge_submission_20__score=0.2952258507364594.csv
Processing ../submissions/logreg_merge_submission_20__score=0.2952258507364594.csv


,track,prediction
0,17730,"0.040445685041552104,0.06861758318313682,0.365..."
1,32460,"0.3565789677636127,0.7216522928735407,0.209785..."
2,11288,"0.3548748528396384,0.5866811172759285,0.037482..."
3,18523,"0.35118102736135415,0.2979960494632946,0.07774..."
4,71342,"0.4062862939135171,0.19521934536442984,0.01500..."
...,...,...
25575,40192,"0.2842964287248602,0.9328929199190559,0.388727..."
25576,4481,"0.7085207032308022,0.5269353714858608,0.046514..."
25577,5257,"0.3456539545590188,0.10950912088410736,0.00933..."
25578,73095,"0.4971107535262388,0.5080881060822203,0.001891..."


In [112]:
merge_subm([
    "../submissions/merge_submission_18__score=0.2952556100056696.csv",
    "../submissions/logreg_merge_submission_20__score=0.2952258507364594.csv",
    "../submissions/mlp_merge_submission_20__score=0.2952258507364594.csv",
],weights=[0.85, 0.09, 0.06], outname="../submissions/merge_submission_24.csv", scale=True)

Processing ../submissions/merge_submission_18__score=0.2952556100056696.csv
Processing ../submissions/logreg_merge_submission_20__score=0.2952258507364594.csv
Processing ../submissions/mlp_merge_submission_20__score=0.2952258507364594.csv


,track,prediction
0,17730,"0.13759506413633527,0.16270997570407203,0.2639..."
1,32460,"0.20922528837392418,0.5769940020022952,0.47452..."
2,11288,"0.26133651437379446,0.3312165593272151,0.23431..."
3,18523,"0.5869398648319716,0.12927777811813068,0.36067..."
4,71342,"0.08153940810204435,0.025976481459137002,0.078..."
...,...,...
25575,40192,"0.3174909765781052,0.8404715396361375,0.732375..."
25576,4481,"0.9409006358938784,0.20103958629064905,0.13118..."
25577,5257,"0.12088541411814524,0.11369613086899488,0.0877..."
25578,73095,"0.9949886802969392,0.07188502667967084,0.05223..."


In [120]:
merge_subm([
    "../submissions/merge_submission_24__score=0.29598460268633153.csv",
    "../submissions/logreg_stacking.csv", 
],weights=[0.9, 0.1], outname="../submissions/merge_submission_26.csv", scale=True)

Processing ../submissions/merge_submission_24__score=0.29598460268633153.csv
Processing ../submissions/logreg_stacking.csv


,track,prediction
0,17730,"0.1315927194123646,0.17555504329589386,0.27692..."
1,32460,"0.20126104012989876,0.5832729054457169,0.49165..."
2,11288,"0.2502143294236784,0.3477524400550658,0.247657..."
3,18523,"0.5858071183864006,0.13603494517368758,0.37705..."
4,71342,"0.07820052161998903,0.023787882203661348,0.079..."
...,...,...
25575,40192,"0.3039709412193895,0.8461154911255333,0.739423..."
25576,4481,"0.9393620012373052,0.21352499906052794,0.13476..."
25577,5257,"0.11525916175541734,0.1187788041125094,0.08813..."
25578,73095,"0.9948417844399198,0.06946853610133535,0.05154..."


In [124]:
merge_subm([
    "../submissions/merge_submission_24__score=0.29598460268633153.csv",
    "../submissions/merge_submission_4__score=0.28771512798027166.csv", 
],weights=[0.8, 0.2], outname="../submissions/merge_submission_26.csv", scale=True)

Processing ../submissions/merge_submission_24__score=0.29598460268633153.csv
Processing ../submissions/merge_submission_4__score=0.28771512798027166.csv


,track,prediction
0,17730,"0.12020668604814594,0.14553704118236988,0.2510..."
1,32460,"0.18955049403144053,0.5734621116552514,0.46098..."
2,11288,"0.2483307026476689,0.30952869096627483,0.22161..."
3,18523,"0.5736595068425289,0.11236690614762745,0.34630..."
4,71342,"0.07163817356715652,0.02022451611445447,0.0682..."
...,...,...
25575,40192,"0.30657002792262944,0.8413592001886757,0.72530..."
25576,4481,"0.9406771525390385,0.18602583638698983,0.11461..."
25577,5257,"0.10657552853994873,0.09912649545928648,0.0762..."
25578,73095,"0.9949887629933011,0.0570780530283001,0.043187..."


In [6]:
merge([62, 63, 64, 65, 66], score_threshold=-2, fold_idx=range(1000))

Processing prediction__cfg=62__fold_idx=1__epoch=21__score=0.25136.csv
Processing prediction__cfg=62__fold_idx=0__epoch=15__score=0.25553.csv
Processing prediction__cfg=62__fold_idx=2__epoch=15__score=0.25666.csv
Processing prediction__cfg=63__fold_idx=1__epoch=14__score=0.25743.csv
Processing prediction__cfg=63__fold_idx=3__epoch=16__score=0.26106.csv
Processing prediction__cfg=63__fold_idx=3__epoch=17__score=0.26151.csv
Processing prediction__cfg=63__fold_idx=2__epoch=21__score=0.25382.csv
Processing prediction__cfg=63__fold_idx=4__epoch=14__score=0.25559.csv
Processing prediction__cfg=63__fold_idx=4__epoch=13__score=0.25526.csv
Processing prediction__cfg=63__fold_idx=0__epoch=21__score=0.25256.csv
Processing prediction__cfg=63__fold_idx=1__epoch=15__score=0.25756.csv
Processing prediction__cfg=64__fold_idx=3__epoch=13__score=0.25547.csv
Processing prediction__cfg=64__fold_idx=2__epoch=15__score=0.26244.csv
Processing prediction__cfg=64__fold_idx=1__epoch=17__score=0.25032.csv
Proces

,track,prediction
0,17730,"0.2571341647916667,0.3040500508333333,0.374426..."
1,32460,"0.37070862624999995,0.5881787624999999,0.54924..."
2,11288,"0.40421195458333337,0.45427472333333335,0.3675..."
3,18523,"0.6144113045833334,0.2590396108333333,0.440073..."
4,71342,"0.1697508854166667,0.108956035375,0.1792123758..."
...,...,...
25575,40192,"0.4352825529166666,0.7868904866666666,0.639567..."
25576,4481,"0.9022781770833332,0.3417213616666666,0.272760..."
25577,5257,"0.24318588345833336,0.24023626020833325,0.1695..."
25578,73095,"0.9862373208333337,0.17977330220833332,0.13562..."


In [10]:
pd.read_csv("../submissions/merge_submission_24__score=0.29598460268633153.csv")

,track,prediction
0,17730,"0.13759506413633527,0.16270997570407203,0.2639..."
1,32460,"0.20922528837392418,0.5769940020022952,0.47452..."
2,11288,"0.26133651437379446,0.3312165593272151,0.23431..."
3,18523,"0.5869398648319716,0.12927777811813068,0.36067..."
4,71342,"0.08153940810204435,0.025976481459137002,0.078..."
...,...,...
25575,40192,"0.3174909765781052,0.8404715396361375,0.732375..."
25576,4481,"0.9409006358938784,0.20103958629064905,0.13118..."
25577,5257,"0.12088541411814524,0.11369613086899488,0.0877..."
25578,73095,"0.9949886802969392,0.07188502667967084,0.05223..."


In [12]:
merge_subm([
    "../submissions/merge_submission_24__score=0.29598460268633153.csv",
    "../submissions/submission__cfg=62_63_64_65_66__score_threshold=-2.csv", 
], weights=[0.89, 0.11], outname="../submissions/merge_submission_27.csv", scale=True)

Processing ../submissions/merge_submission_24__score=0.29598460268633153.csv
Processing ../submissions/submission__cfg=62_63_64_65_66__score_threshold=-2.csv


,track,prediction
0,17730,"0.1447039956751,0.16917686591822642,0.27959959..."
1,32460,"0.22270289797635026,0.5726189170703746,0.49288..."
2,11288,"0.2729262624918203,0.33800900636691616,0.25198..."
3,18523,"0.5868483132648337,0.1336024887448583,0.375579..."
4,71342,"0.08386309008322868,0.022178193475066388,0.086..."
...,...,...
25575,40192,"0.3263783523710574,0.8316620348626496,0.737269..."
25576,4481,"0.9359906981579169,0.20805878159032806,0.14629..."
25577,5257,"0.12817418319282192,0.11731955882612283,0.0932..."
25578,73095,"0.9946486468536193,0.0722593775312636,0.056632..."


In [15]:
merge_subm([
    os.path.join("../submissions/", x) for x in os.listdir("../submissions/")
], outname="../submissions/merge_submission_28.csv", scale=True)

Processing ../submissions/merge_submission_all__score=0.2802971642515184.csv
Processing ../submissions/logreg_merge_submission_20__score=0.2952258507364594.csv
Processing ../submissions/submission__cfg=31__score_threshold=0__score=0.27203014053591923.csv
Processing ../submissions/prediction_epoch_15_nan_deberta.csv
Processing ../submissions/merge_submission__score=0.2809188604719804.csv
Processing ../submissions/submission__cfg=35__score_threshold=0__score=0.27698060624865495.csv
Processing ../submissions/submission__cfg=60_61__score_threshold=-2.csv
Processing ../submissions/logreg_stacking__score=0.26850270506761054.csv
Processing ../submissions/submission__cfg=6.csv
Processing ../submissions/merge_submission_5__score=0.28896974822665383.csv
Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv
Processing ../submissions/mlp_merge_submission_20__score=0.2952258507364594.csv
Processing ../submissions/submission__cfg=43_44_45_46__score_threshold=-2__score=0.2822711

,track,prediction
0,17730,"0.17503647923823623,0.20008810918489098,0.3270..."
1,32460,"0.2568138031947835,0.5820593750289994,0.516084..."
2,11288,"0.3123256418770167,0.3685128718382894,0.289988..."
3,18523,"0.594967407723659,0.16374249112937245,0.408136..."
4,71342,"0.10721688071897884,0.024427467710462725,0.098..."
...,...,...
25575,40192,"0.3684552223286345,0.8206433062131724,0.743593..."
25576,4481,"0.9223481475194539,0.25432621938571404,0.17283..."
25577,5257,"0.16171058936922295,0.15163323445963256,0.1151..."
25578,73095,"0.9922871556739072,0.09642481586872215,0.06630..."


In [32]:
merge_subm([
    os.path.join("../submissions/", x) for x in os.listdir("../submissions/")
], outname="../submissions/merge_submission_29.csv", scale=False, rank=True)

Processing ../submissions/merge_submission_all__score=0.2802971642515184.csv
Processing ../submissions/logreg_merge_submission_20__score=0.2952258507364594.csv
Processing ../submissions/merge_submission_27__score=0.2954113579821891.csv
Processing ../submissions/submission__cfg=31__score_threshold=0__score=0.27203014053591923.csv
Processing ../submissions/prediction_epoch_15_nan_deberta.csv
Processing ../submissions/merge_submission__score=0.2809188604719804.csv
Processing ../submissions/submission__cfg=35__score_threshold=0__score=0.27698060624865495.csv
Processing ../submissions/submission__cfg=60_61__score_threshold=-2.csv
Processing ../submissions/logreg_stacking__score=0.26850270506761054.csv
Processing ../submissions/submission__cfg=6.csv
Processing ../submissions/merge_submission_5__score=0.28896974822665383.csv
Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv
Processing ../submissions/mlp_merge_submission_20__score=0.2952258507364594.csv
Processing ../

,track,prediction
0,17730,"0.2300039093041438,0.33506710972113624,0.69251..."
1,32460,"0.37697289549126917,0.68550625488663,0.8742839..."
2,11288,"0.4512242637477194,0.5658157414646859,0.631629..."
3,18523,"0.651890800104248,0.2499244201198853,0.7828792..."
4,71342,"0.10695139431847797,0.012804274172530621,0.178..."
...,...,...
25575,40192,"0.5068386760489968,0.7928700807922854,0.962376..."
25576,4481,"0.8273905394839719,0.4360893927547564,0.359006..."
25577,5257,"0.20922335157675268,0.22442533229085226,0.1972..."
25578,73095,"0.9712809486578055,0.054886630179827985,0.0658..."


In [43]:
merge_subm([
    "../submissions/logreg_merge_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv", 
    "../submissions/merge_submission_24__score=0.29598460268633153.csv"
], outname="../submissions/merge_submission_30.csv", weights=[0.8, 0.2])

Processing ../submissions/logreg_merge_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv
Processing ../submissions/merge_submission_24__score=0.29598460268633153.csv


,track,prediction
0,17730,"0.04645912262511377,0.06539622348381438,0.1769..."
1,32460,"0.10384126587208095,0.5816296224943692,0.33183..."
2,11288,"0.1749323549009792,0.3005086556755213,0.094690..."
3,18523,"0.6198126114857042,0.056083340372346795,0.3029..."
4,71342,"0.023470726426007373,0.006308617062598831,0.02..."
...,...,...
25575,40192,"0.20499130896142603,0.9124181321425553,0.62739..."
25576,4481,"0.9647249039261232,0.14771807802693135,0.05459..."
25577,5257,"0.04505393024059914,0.03649543357521695,0.0209..."
25578,73095,"0.9979800874604186,0.02228418652166697,0.01209..."


In [44]:
merge_subm([
    "../submissions/logreg_merge_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv", 
    "../submissions/merge_submission_24__score=0.29598460268633153.csv"
], outname="../submissions/merge_submission_31.csv", weights=[0.95, 0.05])

Processing ../submissions/logreg_merge_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv
Processing ../submissions/merge_submission_24__score=0.29598460268633153.csv


,track,prediction
0,17730,"0.029371133591759732,0.047149894942516056,0.16..."
1,32460,"0.08408176165298534,0.582498801336633,0.305083..."
2,11288,"0.15873157499982632,0.29475092374082873,0.0685..."
3,18523,"0.6259762514832791,0.04235938329501231,0.29217..."
4,71342,"0.012582848611750436,0.0026208924882479235,0.0..."
...,...,...
25575,40192,"0.18389762128329865,0.9259081182375086,0.60771..."
25576,4481,"0.9691919541821689,0.13772029522748425,0.04023..."
25577,5257,"0.03083552701355924,0.022020302832633586,0.008..."
25578,73095,"0.9985409763035709,0.012984028992041244,0.0045..."


In [45]:
merge_subm([
    "../submissions/logreg_merge_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv", 
    "../submissions/merge_submission_24__score=0.29598460268633153.csv"
], outname="../submissions/merge_submission_32.csv", weights=[0.2, 0.8])

Processing ../submissions/logreg_merge_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv
Processing ../submissions/merge_submission_24__score=0.29598460268633153.csv


,track,prediction
0,17730,"0.1148110787585299,0.13838153764900762,0.24220..."
1,32460,"0.18287928274846338,0.5781529071253138,0.43885..."
2,11288,"0.23973547450559063,0.3235395834142917,0.19940..."
3,18523,"0.5951580514954047,0.11097916868168473,0.34624..."
4,71342,"0.0670222376830351,0.021059515360002463,0.0644..."
...,...,...
25575,40192,"0.28936605967393547,0.858458187762742,0.706130..."
25576,4481,"0.9468567029019397,0.18770920922471962,0.11203..."
25577,5257,"0.10192754314875872,0.0943959565455504,0.07109..."
25578,73095,"0.9957365320878091,0.05948481664016987,0.04219..."


In [46]:
merge_subm([
    "../submissions/logreg_merge_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv", 
    "../submissions/merge_submission_24__score=0.29598460268633153.csv"
], outname="../submissions/merge_submission_33.csv", weights=[0.1, 0.9])

Processing ../submissions/logreg_merge_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv
Processing ../submissions/merge_submission_24__score=0.29598460268633153.csv


,track,prediction
0,17730,"0.1262030714474326,0.15054575667653983,0.25307..."
1,32460,"0.19605228556119378,0.5775734545638045,0.45669..."
2,11288,"0.25053599443969254,0.3273780713707534,0.21685..."
3,18523,"0.5910489581636882,0.1201284733999077,0.353459..."
4,71342,"0.07428082289253973,0.02351799840956973,0.0713..."
...,...,...
25575,40192,"0.3034285181260203,0.8494648636994397,0.719252..."
25576,4481,"0.943878669397909,0.19437439775768436,0.121608..."
25577,5257,"0.11140647863345197,0.10404604370727263,0.0794..."
25578,73095,"0.9953626061923742,0.06568492165992035,0.04721..."


In [50]:
merge_subm([
    os.path.join("../submissions/", x) for x in os.listdir("../submissions/") if x.startswith("merge_subm")
], outname="../submissions/merge_submission_34.csv", rank=True)

Processing ../submissions/merge_submission_all__score=0.2802971642515184.csv
Processing ../submissions/merge_submission_27__score=0.2954113579821891.csv
Processing ../submissions/merge_submission__score=0.2809188604719804.csv
Processing ../submissions/merge_submission_5__score=0.28896974822665383.csv
Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv
Processing ../submissions/merge_submission_33__score=0.2960919791783309.csv
Processing ../submissions/merge_submission_16__score=0.28969047017154304.csv
Processing ../submissions/merge_submission_18__score=0.2952556100056696.csv
Processing ../submissions/merge_submission_31__score=0.27749068698074497.csv
Processing ../submissions/merge_submission_10__score=0.2900969049214374.csv
Processing ../submissions/merge_submission_15__score=0.289886916346524.csv
Processing ../submissions/merge_submission_28__score=0.29320377002012604.csv
Processing ../submissions/merge_submission_23__score=0.2946765556763446.csv
Processing .

,track,prediction
0,17730,"0.22327645679069122,0.340702754909626,0.697036..."
1,32460,"0.3721496573609897,0.6928425240307226,0.871577..."
2,11288,"0.4566469668399025,0.558225635836821,0.6427735..."
3,18523,"0.6457319597111713,0.25552706618221965,0.79690..."
4,71342,"0.10421514970335279,0.008495837740882126,0.165..."
...,...,...
25575,40192,"0.5056615922365819,0.7979430161431267,0.965248..."
25576,4481,"0.8296267764337948,0.4390125557650738,0.355274..."
25577,5257,"0.2026376305017707,0.21788736604884332,0.19311..."
25578,73095,"0.9749045669870763,0.05234903187232672,0.06034..."


In [52]:
merge_subm([
    "../submissions/merge_submission_33__score=0.2960919791783309.csv", 
    "../submissions/merge_submission_34.csv"
], outname="../submissions/merge_submission_35.csv", weights=[0.8, 0.2], rank=True)

Processing ../submissions/merge_submission_33__score=0.2960919791783309.csv
Processing ../submissions/merge_submission_34.csv


,track,prediction
0,17730,"0.23187646598905398,0.33748240813135266,0.6851..."
1,32460,"0.37751368256450357,0.6895152462861611,0.87182..."
2,11288,"0.44983580922595784,0.563268178264269,0.624276..."
3,18523,"0.6562861610633308,0.24935105551211886,0.79891..."
4,71342,"0.09351055512118843,0.0012587959343236905,0.14..."
...,...,...
25575,40192,"0.4985457388584832,0.7971305707584051,0.966982..."
25576,4481,"0.8336356528537922,0.4270367474589523,0.345715..."
25577,5257,"0.1954182955433933,0.19877247849882723,0.17563..."
25578,73095,"0.9789053948397185,0.05278342455043003,0.06078..."


In [ ]:
merge_subm([
    "../submissions/merge_submission_33__score=0.2960919791783309.csv", 
    "../submissions/merge_submission_34.csv"
], outname="../submissions/merge_submission_35.csv", weights=[0.7, 0.2], rank=True)

In [212]:
submissions_to_merge = [
    os.path.join("../submissions/", x) 
    for x in os.listdir("../submissions/") 
    if x.startswith("merge")
]
submissions_weights = [
    float(x.rsplit("__score=", 1)[1].rsplit(".", 1)[0])
    for x in os.listdir("../submissions/") 
    if x.startswith("merge")
]

IndexError: list index out of range

In [69]:
merge_subm(
    submissions_to_merge, 
    outname="../submissions/merge_submission_36.csv", 
    weights=submissions_weights,
    rank=True
)

Processing ../submissions/merge_submission_all__score=0.2802971642515184.csv
Processing ../submissions/merge_submission_27__score=0.2954113579821891.csv
Processing ../submissions/merge_submission__score=0.2809188604719804.csv
Processing ../submissions/merge_submission_34__score=0.29388558315457386.csv
Processing ../submissions/merge_submission_5__score=0.28896974822665383.csv
Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv
Processing ../submissions/merge_submission_33__score=0.2960919791783309.csv
Processing ../submissions/merge_submission_16__score=0.28969047017154304.csv
Processing ../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv
Processing ../submissions/merge_submission_18__score=0.2952556100056696.csv
Processing ../submissions/merge_submission_31__score=0.27749068698074497.csv
Processing ../submissions/merge_submission_10__score=0.2900969049214374.csv
Processing ../submissions/merge_submission_15__score

,track,prediction
0,17730,"0.22330558594171285,0.34062977540764255,0.6974..."
1,32460,"0.3735617314309142,0.6925881057227544,0.870575..."
2,11288,"0.45527618041308404,0.5596266266854427,0.63760..."
3,18523,"0.6476005023763386,0.2567862465259053,0.796993..."
4,71342,"0.10684623848194254,0.008634087862369879,0.168..."
...,...,...
25575,40192,"0.5049796563823215,0.8000831710381321,0.964440..."
25576,4481,"0.8283583865268688,0.43977197459806205,0.35604..."
25577,5257,"0.20302270069206332,0.21607942034309383,0.1877..."
25578,73095,"0.9723822289545955,0.05226803077638953,0.06038..."


In [91]:
merge_subm([
    "../submissions/merge_submission_33__score=0.2960919791783309.csv", 
    "../submissions/merge_submission_36__score=0.2940615791568242.csv"
], outname="../submissions/merge_submission_37.csv", weights=[0.5, 0.5], scale=True)

Processing ../submissions/merge_submission_33__score=0.2960919791783309.csv
Processing ../submissions/merge_submission_36__score=0.2940615791568242.csv


,track,prediction
0,17730,"0.17845212615028389,0.24894813394053,0.4292969..."
1,32460,"0.2924510167060967,0.6381726988934363,0.621878..."
2,11288,"0.3619530166124266,0.4499732999179644,0.383539..."
3,18523,"0.6221109251835139,0.18925185001303546,0.52987..."
4,71342,"0.0911029080070247,0.009519112847877653,0.1089..."
...,...,...
25575,40192,"0.41321455816556885,0.8225666974010966,0.81889..."
25576,4481,"0.881543162327007,0.32317770474304597,0.213928..."
25577,5257,"0.16062149671984283,0.15965994368261976,0.1215..."
25578,73095,"0.983776381993104,0.05278359120488568,0.051423..."


In [96]:
merge(range(58, 72), score_threshold=-2, fold_idx=range(-2,100), scale=True)

Processing prediction__cfg=58__fold_idx=0__epoch=15__score=0.25944.csv
Processing prediction__cfg=58__fold_idx=3__epoch=12__score=0.26978.csv
Processing prediction__cfg=58__fold_idx=1__epoch=12__score=0.26408.csv
Processing prediction__cfg=58__fold_idx=-1__epoch=18__score=-1.csv
Processing prediction__cfg=58__fold_idx=3__epoch=13__score=0.27106.csv
Processing prediction__cfg=58__fold_idx=0__epoch=16__score=0.25950.csv
Processing prediction__cfg=58__fold_idx=4__epoch=12__score=0.26145.csv
Processing prediction__cfg=58__fold_idx=2__epoch=14__score=0.26187.csv
Processing prediction__cfg=58__fold_idx=2__epoch=16__score=0.26196.csv
Processing prediction__cfg=58__fold_idx=0__epoch=18__score=0.25964.csv
Processing prediction__cfg=58__fold_idx=1__epoch=13__score=0.26430.csv
Processing prediction__cfg=58__fold_idx=4__epoch=13__score=0.26331.csv
Processing prediction__cfg=58__fold_idx=-1__epoch=19__score=-1.csv
Processing prediction__cfg=59__fold_idx=0__epoch=17__score=0.25504.csv
Processing pre

,track,prediction
0,17730,"0.2173938269976357,0.24414243904505278,0.43039..."
1,32460,"0.3451982560360958,0.5466672883831956,0.653539..."
2,11288,"0.3825991126205619,0.42502294415818787,0.40254..."
3,18523,"0.6153311151463983,0.2002933506415649,0.506696..."
4,71342,"0.108383502310747,0.007709265872628435,0.13884..."
...,...,...
25575,40192,"0.41878503980625165,0.7691621008214553,0.79079..."
25576,4481,"0.8868824178104333,0.30721050154333507,0.27471..."
25577,5257,"0.20887364412261827,0.16661346020711648,0.1445..."
25578,73095,"0.9919617077307847,0.1073456206337065,0.096258..."


In [99]:
merge_subm([
    os.path.join("../predictions_test", x) for x in os.listdir("../predictions_test") 
], 
    outname="../submissions/merge_submission_38.csv",
    rank=True,
)

Processing ../predictions_test/prediction__cfg=58__fold_idx=0__epoch=15__score=0.25944.csv
Processing ../predictions_test/prediction__cfg=44__fold_idx=0__epoch=18__score=0.25767.csv
Processing ../predictions_test/prediction__cfg=42__fold_idx=1__epoch=17__score=0.25192.csv
Processing ../predictions_test/prediction__cfg=71__fold_idx=0__epoch=14__score=0.26265.csv
Processing ../predictions_test/prediction__cfg=5__fold_idx=2__epoch=17__score=0.24446.csv
Processing ../predictions_test/prediction__cfg=44__fold_idx=2__epoch=18__score=0.25624.csv
Processing ../predictions_test/prediction__cfg=38__fold_idx=-1__epoch=18__score=-1.csv
Processing ../predictions_test/prediction__cfg=37__fold_idx=1__epoch=15__score=0.26000.csv
Processing ../predictions_test/prediction__cfg=59__fold_idx=0__epoch=17__score=0.25504.csv
Processing ../predictions_test/prediction__cfg=48__fold_idx=0__epoch=13__score=0.24586.csv
Processing ../predictions_test/prediction__cfg=61__fold_idx=2__epoch=12__score=0.25703.csv
Proc

Processing ../predictions_test/prediction__cfg=6__fold_idx=1__epoch=10__score=0.23721.csv
Processing ../predictions_test/prediction__cfg=69__fold_idx=0__epoch=17__score=0.26713.csv
Processing ../predictions_test/prediction__cfg=58__fold_idx=0__epoch=16__score=0.25950.csv
Processing ../predictions_test/prediction__cfg=71__fold_idx=1__epoch=18__score=0.25973.csv
Processing ../predictions_test/prediction__cfg=29__fold_idx=1__epoch=19__score=0.25337.csv
Processing ../predictions_test/prediction__cfg=22__fold_idx=1__epoch=13__score=0.24587.csv
Processing ../predictions_test/prediction__cfg=31__fold_idx=0__epoch=26__score=0.25486.csv
Processing ../predictions_test/prediction__cfg=60__fold_idx=0__epoch=18__score=0.26564.csv
Processing ../predictions_test/prediction__cfg=22__fold_idx=0__epoch=14__score=0.24697.csv
Processing ../predictions_test/prediction__cfg=4__fold_idx=0__epoch=14__score=0.22906.csv
Processing ../predictions_test/prediction__cfg=68__fold_idx=0__epoch=14__score=0.26220.csv
P

Processing ../predictions_test/prediction__cfg=24__fold_idx=-1__epoch=22__score=-1.csv
Processing ../predictions_test/prediction__cfg=70__fold_idx=-1__epoch=19__score=-1.csv
Processing ../predictions_test/prediction__cfg=21__fold_idx=1__epoch=26__score=0.26122.csv
Processing ../predictions_test/prediction__cfg=31__fold_idx=1__epoch=28__score=0.25363.csv
Processing ../predictions_test/prediction__cfg=8__fold_idx=-1__epoch=10__score=-1.csv
Processing ../predictions_test/prediction__cfg=7__fold_idx=0__epoch=9__score=0.23697.csv
Processing ../predictions_test/prediction__cfg=21__fold_idx=2__epoch=19__score=0.26701.csv
Processing ../predictions_test/prediction__cfg=60__fold_idx=-2__epoch=1__score=-2.csv
Processing ../predictions_test/prediction__cfg=30__fold_idx=-1__epoch=24__score=-1.csv
Processing ../predictions_test/prediction__cfg=29__fold_idx=0__epoch=14__score=0.25171.csv
Processing ../predictions_test/prediction__cfg=5__fold_idx=0__epoch=14__score=0.23977.csv
Processing ../prediction

Processing ../predictions_test/prediction__cfg=49__fold_idx=2__epoch=12__score=0.24754.csv
Processing ../predictions_test/prediction__cfg=4__fold_idx=2__epoch=15__score=0.23043.csv
Processing ../predictions_test/prediction__cfg=61__fold_idx=1__epoch=11__score=0.25960.csv
Processing ../predictions_test/prediction__cfg=68__fold_idx=1__epoch=13__score=0.26818.csv
Processing ../predictions_test/prediction__cfg=44__fold_idx=-1__epoch=19__score=-1.csv
Processing ../predictions_test/prediction__cfg=53__fold_idx=-1__epoch=19__score=-1.csv
Processing ../predictions_test/prediction__cfg=47__fold_idx=0__epoch=16__score=0.25637.csv
Processing ../predictions_test/prediction__cfg=60__fold_idx=1__epoch=13__score=0.26613.csv
Processing ../predictions_test/prediction__cfg=69__fold_idx=3__epoch=12__score=0.26090.csv
Processing ../predictions_test/prediction__cfg=45__fold_idx=1__epoch=16__score=0.25425.csv
Processing ../predictions_test/prediction__cfg=43__fold_idx=-1__epoch=19__score=-1.csv
Processing .

Processing ../predictions_test/prediction__cfg=63__fold_idx=0__epoch=21__score=0.25256.csv
Processing ../predictions_test/prediction__cfg=7__fold_idx=1__epoch=13__score=0.23426.csv
Processing ../predictions_test/prediction__cfg=69__fold_idx=-1__epoch=19__score=-1.csv
Processing ../predictions_test/prediction__cfg=31__fold_idx=2__epoch=17__score=0.25841.csv
Processing ../predictions_test/prediction__cfg=64__fold_idx=0__epoch=15__score=0.26178.csv
Processing ../predictions_test/prediction__cfg=62__fold_idx=-2__epoch=1__score=-2.csv
Processing ../predictions_test/prediction__cfg=19__fold_idx=0__epoch=16__score=0.24880.csv
Processing ../predictions_test/prediction__cfg=69__fold_idx=1__epoch=13__score=0.26408.csv
Processing ../predictions_test/prediction__cfg=4__fold_idx=2__epoch=16__score=0.23021.csv
Processing ../predictions_test/prediction__cfg=6__fold_idx=0__epoch=10__score=0.23640.csv
Processing ../predictions_test/prediction__cfg=49__fold_idx=-1__epoch=19__score=-1.csv
Processing ../p

,track,prediction
0,17730,"0.24338039510994677,0.3320299917490265,0.67973..."
1,32460,"0.40790742335421903,0.6598396464691957,0.88455..."
2,11288,"0.44589342912629754,0.5753632770417547,0.61591..."
3,18523,"0.6631701213865958,0.2589790591007519,0.761508..."
4,71342,"0.11375876891148397,0.01642015831781022,0.1671..."
...,...,...
25575,40192,"0.5069327996022241,0.7861802495469173,0.953553..."
25576,4481,"0.8274803363803084,0.4038904782322018,0.356081..."
25577,5257,"0.21234439348137044,0.195957923637058,0.196428..."
25578,73095,"0.9736071527655173,0.06947004608294931,0.08088..."


In [103]:
merge_subm([
    "../submissions/merge_submission_38__score=0.2861704325134863.csv",
    "../submissions/merge_submission_33__score=0.2960919791783309.csv"
], weights=[0.25, 0.75], scale=False, rank=False, outname="../submissions/merge_submission_39.csv")

Processing ../submissions/merge_submission_38__score=0.2861704325134863.csv
Processing ../submissions/merge_submission_33__score=0.2960919791783309.csv


,track,prediction
0,17730,"0.15549740236306114,0.1959168154446615,0.35974..."
1,32460,"0.24901607000945009,0.5981400025401523,0.56365..."
2,11288,"0.2993753531113438,0.38937437278850373,0.31662..."
3,18523,"0.6090792489694151,0.15484111982511875,0.45547..."
4,71342,"0.08415030939727579,0.021743538386629853,0.095..."
...,...,...
25575,40192,"0.35430458849507124,0.8336437101613091,0.77782..."
25576,4481,"0.9147790861435089,0.24675341787631372,0.18022..."
25577,5257,"0.13664095734543158,0.12702401368971897,0.1086..."
25578,73095,"0.98992374283566,0.06663120276567759,0.0556330..."


In [106]:
merge_subm([
    "../submissions/merge_submission_38__score=0.2861704325134863.csv",
    "../submissions/merge_submission_33__score=0.2960919791783309.csv"
], weights=[0.15, 0.85], scale=False, rank=False, outname="../submissions/merge_submission_40.csv")

Processing ../submissions/merge_submission_38__score=0.2861704325134863.csv
Processing ../submissions/merge_submission_33__score=0.2960919791783309.csv


,track,prediction
0,17730,"0.14377966999680972,0.1777683919374128,0.31707..."
1,32460,"0.22783055623014756,0.5899133833496132,0.52086..."
2,11288,"0.2798396096426833,0.3645758522214036,0.276717..."
3,18523,"0.6018671326471243,0.14095606125503435,0.41466..."
4,71342,"0.08020251479538136,0.022453322395805806,0.085..."
...,...,...
25575,40192,"0.33395416034745085,0.8399721715765613,0.75439..."
25576,4481,"0.9264189194452689,0.225801809828862,0.1567792..."
25577,5257,"0.12654716586063974,0.11783282569674043,0.0969..."
25578,73095,"0.9920992881783456,0.06625269032337469,0.05226..."


In [114]:
submissions_to_merge = [
    os.path.join("../predictions_test/", x) 
    for x in os.listdir("../predictions_test/") 
    if float(x.rsplit("__score=", 1)[1].rsplit(".", 1)[0]) > 0.25
]
submissions_weights = [
    float(x.rsplit("__score=", 1)[1].rsplit(".", 1)[0])
    for x in os.listdir("../predictions_test/") 
    if float(x.rsplit("__score=", 1)[1].rsplit(".", 1)[0]) > 0.25
]

In [115]:
assert len(submissions_to_merge) == len(submissions_weights)

In [116]:
merge_subm(
    submissions_to_merge, 
    outname="../submissions/merge_submission_41.csv", 
    weights=submissions_weights,
    rank=True
)

Processing ../predictions_test/prediction__cfg=58__fold_idx=0__epoch=15__score=0.25944.csv
Processing ../predictions_test/prediction__cfg=44__fold_idx=0__epoch=18__score=0.25767.csv
Processing ../predictions_test/prediction__cfg=42__fold_idx=1__epoch=17__score=0.25192.csv
Processing ../predictions_test/prediction__cfg=71__fold_idx=0__epoch=14__score=0.26265.csv
Processing ../predictions_test/prediction__cfg=44__fold_idx=2__epoch=18__score=0.25624.csv
Processing ../predictions_test/prediction__cfg=37__fold_idx=1__epoch=15__score=0.26000.csv
Processing ../predictions_test/prediction__cfg=59__fold_idx=0__epoch=17__score=0.25504.csv
Processing ../predictions_test/prediction__cfg=61__fold_idx=2__epoch=12__score=0.25703.csv
Processing ../predictions_test/prediction__cfg=44__fold_idx=1__epoch=17__score=0.25373.csv
Processing ../predictions_test/prediction__cfg=58__fold_idx=3__epoch=12__score=0.26978.csv
Processing ../predictions_test/prediction__cfg=19__fold_idx=2__epoch=15__score=0.25543.csv

Processing ../predictions_test/prediction__cfg=30__fold_idx=2__epoch=16__score=0.25863.csv
Processing ../predictions_test/prediction__cfg=22__fold_idx=1__epoch=16__score=0.25098.csv
Processing ../predictions_test/prediction__cfg=64__fold_idx=1__epoch=17__score=0.25032.csv
Processing ../predictions_test/prediction__cfg=21__fold_idx=0__epoch=13__score=0.26361.csv
Processing ../predictions_test/prediction__cfg=59__fold_idx=1__epoch=15__score=0.25788.csv
Processing ../predictions_test/prediction__cfg=60__fold_idx=3__epoch=14__score=0.26526.csv
Processing ../predictions_test/prediction__cfg=30__fold_idx=2__epoch=14__score=0.25306.csv
Processing ../predictions_test/prediction__cfg=60__fold_idx=3__epoch=17__score=0.26536.csv
Processing ../predictions_test/prediction__cfg=9__fold_idx=1__epoch=18__score=0.25126.csv
Processing ../predictions_test/prediction__cfg=70__fold_idx=2__epoch=19__score=0.26516.csv
Processing ../predictions_test/prediction__cfg=43__fold_idx=0__epoch=17__score=0.25746.csv


Processing ../predictions_test/prediction__cfg=62__fold_idx=2__epoch=15__score=0.25666.csv
Processing ../predictions_test/prediction__cfg=38__fold_idx=1__epoch=14__score=0.25544.csv
Processing ../predictions_test/prediction__cfg=64__fold_idx=2__epoch=16__score=0.25124.csv
Processing ../predictions_test/prediction__cfg=58__fold_idx=0__epoch=18__score=0.25964.csv
Processing ../predictions_test/prediction__cfg=19__fold_idx=2__epoch=14__score=0.25034.csv
Processing ../predictions_test/prediction__cfg=71__fold_idx=4__epoch=12__score=0.25751.csv
Processing ../predictions_test/prediction__cfg=58__fold_idx=1__epoch=13__score=0.26430.csv
Processing ../predictions_test/prediction__cfg=64__fold_idx=2__epoch=17__score=0.25300.csv
Processing ../predictions_test/prediction__cfg=71__fold_idx=3__epoch=17__score=0.26287.csv
Processing ../predictions_test/prediction__cfg=64__fold_idx=2__epoch=14__score=0.25919.csv
Processing ../predictions_test/prediction__cfg=59__fold_idx=3__epoch=19__score=0.25094.csv

,track,prediction
0,17730,"0.23533883881822615,0.3274194802531513,0.67254..."
1,32460,"0.4174334799397014,0.6584695622564916,0.882460..."
2,11288,"0.4465584294503363,0.5893088094071788,0.601226..."
3,18523,"0.6710845291342389,0.23942211712924547,0.74831..."
4,71342,"0.09152306649278188,0.004127291250874451,0.151..."
...,...,...
25575,40192,"0.5025889561147913,0.7819852790955675,0.962292..."
25576,4481,"0.8221569326730653,0.41263711095660804,0.37355..."
25577,5257,"0.21397115063517044,0.192315650666002,0.168755..."
25578,73095,"0.9755351526778305,0.07739879629324589,0.07516..."


In [119]:
merge_subm([
    "../submissions/merge_submission_41__score=0.28021666987087057.csv",
    "../submissions/merge_submission_33__score=0.2960919791783309.csv"
], weights=[0.1, 0.9], scale=False, rank=False, outname="../submissions/merge_submission_42.csv")

Processing ../submissions/merge_submission_41__score=0.28021666987087057.csv
Processing ../submissions/merge_submission_33__score=0.2960919791783309.csv


,track,prediction
0,17730,"0.13711664818451194,0.16823312903420098,0.2950..."
1,32460,"0.21819040499904455,0.5856630653330732,0.49926..."
2,11288,"0.2701382379407569,0.3535711451743959,0.255295..."
3,18523,"0.5990525152607433,0.1320578377728415,0.392945..."
4,71342,"0.07600504725256395,0.021578927693700203,0.079..."
...,...,...
25575,40192,"0.32334456192489736,0.8427169052390525,0.74355..."
25576,4481,"0.9317064957254246,0.21620066907757673,0.14680..."
25577,5257,"0.12166294583362383,0.11287300440314557,0.0883..."
25578,73095,"0.9933798608409199,0.0668563091232529,0.050011..."


In [ ]:
merge_subm([
    "../submissions/merge_submission_41__score=0.28021666987087057.csv",
    "../submissions/merge_submission_33__score=0.2960919791783309.csv"
], weights=[0.1, 0.9], scale=False, rank=False, outname="../submissions/merge_submission_42.csv")

In [123]:
1 / 256

0.00390625

In [127]:
merge_subm([
    "../submissions/merge_nn_minus_05_submission_33__score=0.2954777008434945.csv",
    "../submissions/merge_submission_33__score=0.2960919791783309.csv"
], weights=[0.5, 0.5], scale=False, rank=False, outname="../submissions/merge_submission_43.csv")

Processing ../submissions/merge_nn_minus_05_submission_33__score=0.2954777008434945.csv
Processing ../submissions/merge_submission_33__score=0.2960919791783309.csv


,track,prediction
0,17730,"0.1262030714474326,0.15054575667653983,0.25307..."
1,32460,"0.19605228556119378,0.5775734545638045,0.45669..."
2,11288,"0.25053599443969254,0.3273780713707534,0.21685..."
3,18523,"0.5910489581636882,0.1201284733999077,0.353459..."
4,71342,"0.07428082289253973,0.02351799840956973,0.0713..."
...,...,...
25575,40192,"0.3034285181260203,0.8494648636994397,0.719252..."
25576,4481,"0.943878669397909,0.19437439775768436,0.121608..."
25577,5257,"0.11140647863345197,0.10404604370727263,0.0794..."
25578,73095,"0.9953626061923742,0.06568492165992035,0.04721..."


In [129]:
merge_subm([
    "../submissions/merge_nn_minus_05_submission_33__score=0.2954777008434945.csv",
    "../submissions/merge_submission_33__score=0.2960919791783309.csv"
], weights=[0.3, 0.7], scale=False, rank=False, outname="../submissions/merge_submission_44.csv")

Processing ../submissions/merge_nn_minus_05_submission_33__score=0.2954777008434945.csv
Processing ../submissions/merge_submission_33__score=0.2960919791783309.csv


,track,prediction
0,17730,"0.1262030714474326,0.15054575667653983,0.25307..."
1,32460,"0.19605228556119375,0.5775734545638045,0.45669..."
2,11288,"0.25053599443969254,0.32737807137075337,0.2168..."
3,18523,"0.5910489581636882,0.12012847339990769,0.35345..."
4,71342,"0.07428082289253973,0.02351799840956973,0.0713..."
...,...,...
25575,40192,"0.30342851812602023,0.8494648636994397,0.71925..."
25576,4481,"0.9438786693979089,0.19437439775768434,0.12160..."
25577,5257,"0.11140647863345196,0.10404604370727262,0.0794..."
25578,73095,"0.995362606192374,0.06568492165992035,0.047216..."


In [132]:
merge_subm([
    "../submissions/merge_nn_minus_068_submission_33__score=0.2958745289694904.csv",
    "../submissions/merge_submission_33__score=0.2960919791783309.csv"
], weights=[0.5, 0.5], scale=False, rank=True, outname="../submissions/merge_submission_45.csv")

Processing ../submissions/merge_nn_minus_068_submission_33__score=0.2958745289694904.csv
Processing ../submissions/merge_submission_33__score=0.2960919791783309.csv


,track,prediction
0,17730,"0.23405003909304145,0.33620015637216577,0.6818..."
1,32460,"0.37869429241595,0.6884284597341673,0.87157935..."
2,11288,"0.44784988272087567,0.5640734949179046,0.61939..."
3,18523,"0.6587959343236904,0.24765441751368256,0.79906..."
4,71342,"0.091164972634871,0.0006645817044566067,0.1403..."
...,...,...
25575,40192,"0.4965598123534011,0.796989835809226,0.9672009..."
25576,4481,"0.8346755277560595,0.4235731039874902,0.343432..."
25577,5257,"0.1936278342455043,0.19409695074276778,0.17177..."
25578,73095,"0.9792806880375293,0.05437842064112588,0.06153..."


In [135]:
merge_subm(
    [
        "../submissions/logreg_stacking__score=0.26850270506761054.csv",
        "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
        "../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv",
        "../submissions/merge_nn_all_submission_33__score=0.26964665034419966.csv",
        "../submissions/merge_submission_12__score=0.2833985712331001.csv"
    ],
    rank=True, 
    outname="../submissions/merge_submission_46.csv"
)

Processing ../submissions/logreg_stacking__score=0.26850270506761054.csv
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv
Processing ../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv
Processing ../submissions/merge_nn_all_submission_33__score=0.26964665034419966.csv
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv


,track,prediction
0,17730,"0.24899921813917122,0.363182173573104,0.658131..."
1,32460,"0.3866536356528538,0.6733541829554339,0.875746..."
2,11288,"0.43797498045347927,0.5694370602032838,0.60659..."
3,18523,"0.675285379202502,0.2526348709929632,0.7986786..."
4,71342,"0.14948397185301016,0.009194683346364347,0.192..."
...,...,...
25575,40192,"0.47778733385457395,0.8128146989835809,0.95689..."
25576,4481,"0.828436278342455,0.40608287724784986,0.342447..."
25577,5257,"0.22216575449569978,0.1845504300234558,0.15741..."
25578,73095,"0.9550039093041439,0.05480844409695074,0.05963..."


In [140]:
merge_subm(
    [
        "../submissions/merge_submission_46__score=0.2906475981046778.csv",
        "../submissions/merge_submission_33__score=0.2960919791783309.csv"
    ],
    rank=True, 
    weights=[0.4, 0.6],
    outname="../submissions/merge_submission_47.csv"
)

Processing ../submissions/merge_submission_46__score=0.2906475981046778.csv
Processing ../submissions/merge_submission_33__score=0.2960919791783309.csv


,track,prediction
0,17730,"0.23961688819390148,0.34811571540265834,0.6739..."
1,32460,"0.3825879593432369,0.6827834245504301,0.874456..."
2,11288,"0.44519155590304926,0.5672634870992963,0.61579..."
3,18523,"0.6658483189992181,0.24937451133698202,0.80050..."
4,71342,"0.11219702892885067,0.0012275215011727913,0.15..."
...,...,...
25575,40192,"0.48952306489444875,0.8016497263487099,0.96408..."
25576,4481,"0.8312197028928852,0.4177716966379984,0.342587..."
25577,5257,"0.20444878811571543,0.1895621579358874,0.16365..."
25578,73095,"0.9742611415168101,0.050125097732603596,0.0575..."


In [167]:
merge_subm(
    [
        "../submissions/merge_submission_46__score=0.2906475981046778.csv",
        "../submissions/merge_submission_33__score=0.2960919791783309.csv"
    ],
    scale=True, 
    weights=[0.2, 0.8],
    outname="../submissions/merge_submission_48.csv"
)

Processing ../submissions/merge_submission_46__score=0.2906475981046778.csv, 0.2
Processing ../submissions/merge_submission_33__score=0.2960919791783309.csv, 0.8


,track,prediction
0,17730,"0.15488006527201625,0.19338681960577245,0.3113..."
1,32460,"0.24065864019726144,0.5975069737663858,0.51825..."
2,11288,"0.2945298385714224,0.3786233097747099,0.272728..."
3,18523,"0.6118068298235687,0.14426450500461302,0.41808..."
4,71342,"0.09177033712164723,0.013000088759359874,0.088..."
...,...,...
25575,40192,"0.34449936075787324,0.8414715239305659,0.75672..."
25576,4481,"0.918987863819205,0.2374084752602173,0.1528706..."
25577,5257,"0.13725025720922263,0.11607697416583873,0.0901..."
25578,73095,"0.9880526686743599,0.056350441955735575,0.0483..."


In [169]:
merge_subm(
    [
        "../submissions/merge_submission_46__score=0.2906475981046778.csv",
        "../submissions/merge_submission_33__score=0.2960919791783309.csv"
    ],
    scale=True, 
    weights=[0.4, 0.6],
    outname="../submissions/merge_submission_49.csv"
)

Processing ../submissions/merge_submission_46__score=0.2906475981046778.csv, 0.4
Processing ../submissions/merge_submission_33__score=0.2960919791783309.csv, 0.6


,track,prediction
0,17730,"0.18131008993644265,0.2392186880396897,0.37708..."
1,32460,"0.2817472817747107,0.6184610157236774,0.586290..."
2,11288,"0.33499425082242373,0.4308585756958771,0.33594..."
3,18523,"0.6304200879616395,0.17287860577052927,0.49042..."
4,71342,"0.10790838561828497,0.010044411592241731,0.107..."
...,...,...
25575,40192,"0.38220431928887244,0.8341278318429903,0.79487..."
25576,4481,"0.8950080126313182,0.28306263300961887,0.18828..."
25577,5257,"0.1610755974020941,0.13351904782372792,0.10204..."
25578,73095,"0.9802783299165579,0.05415544440248432,0.04950..."


In [175]:
merge_subm(
    [
        "../submissions/merge_submission_46__score=0.2906475981046778.csv",
        "../submissions/merge_submission_33__score=0.2960919791783309.csv"
    ],
    scale=True, 
    weights=[0.15, 0.85],
    outname="../submissions/merge_submission_50.csv"
)

Processing ../submissions/merge_submission_46__score=0.2906475981046778.csv, 0.15
Processing ../submissions/merge_submission_33__score=0.2960919791783309.csv, 0.85


,track,prediction
0,17730,"0.147943254400281,0.18283391803791257,0.296399..."
1,32460,"0.22987453924442305,0.5926096607176006,0.50278..."
2,11288,"0.28390810749930917,0.3660282827843374,0.25834..."
3,18523,"0.6069088009872639,0.13829240223316708,0.40162..."
4,71342,"0.08753482436762877,0.01551066404511836,0.0838..."
...,...,...
25575,40192,"0.33460019019776877,0.8434445746229808,0.74804..."
25576,4481,"0.925254622042442,0.2268125797726642,0.1448145..."
25577,5257,"0.13099713076423264,0.11306972352948841,0.0874..."
25578,73095,"0.9900667958780085,0.058575152497913896,0.0481..."


In [213]:
submissions_to_merge = [
    os.path.join("../submissions/", x) 
    for x in os.listdir("../submissions/") 
    if re.match(r"merge_submission_\d+", x)
]

In [214]:
merge_subm(
    submissions_to_merge,
    scale=True, 
    outname="../submissions/merge_submission_51.csv"
)

Processing ../submissions/merge_submission_27__score=0.2954113579821891.csv, 1
Processing ../submissions/merge_submission_34__score=0.29388558315457386.csv, 1
Processing ../submissions/merge_submission_5__score=0.28896974822665383.csv, 1
Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 1
Processing ../submissions/merge_submission_39__score=0.29576528082337394.csv, 1
Processing ../submissions/merge_submission_38__score=0.2861704325134863.csv, 1
Processing ../submissions/merge_submission_33__score=0.2960919791783309.csv, 1
Processing ../submissions/merge_submission_40__score=0.29589944253546246.csv, 1
Processing ../submissions/merge_submission_41__score=0.28021666987087057.csv, 1
Processing ../submissions/merge_submission_37__score=0.29609181013568253.csv, 1
Processing ../submissions/merge_submission_16__score=0.28969047017154304.csv, 1
Processing ../submissions/merge_submission_18__score=0.2952556100056696.csv, 1
Processing ../submissions/merge_submission_31_

,track,prediction
0,17730,"0.15704680169885238,0.19851134312436264,0.3438..."
1,32460,"0.25092955901269876,0.599798992670103,0.540394..."
2,11288,"0.31278927646556676,0.382236367547864,0.303197..."
3,18523,"0.5974565580058018,0.15295595132116802,0.43787..."
4,71342,"0.0879461536052979,0.013486957812343214,0.0904..."
...,...,...
25575,40192,"0.3664511525027956,0.8295273290772358,0.766619..."
25576,4481,"0.9093861519196069,0.2558300427472143,0.167275..."
25577,5257,"0.14199331301952417,0.12896277110859183,0.1011..."
25578,73095,"0.9881296013168968,0.06073429215217108,0.05162..."


In [219]:
"merge_submission_33__score=0.2960919791783309.csv".rsplit("=", 1)[1].rsplit(".", 1)[0]

'0.2960919791783309'

In [220]:
submissions_to_merge = [
    os.path.join("../submissions/", x) 
    for x in os.listdir("../submissions/") 
    if re.match(r"merge_submission_\d+", x) and float(x.rsplit("=", 1)[1].rsplit(".", 1)[0]) > 0.285
]

In [221]:
merge_subm(
    submissions_to_merge,
    scale=True, 
    outname="../submissions/merge_submission_52.csv"
)

Processing ../submissions/merge_submission_27__score=0.2954113579821891.csv, 1
Processing ../submissions/merge_submission_34__score=0.29388558315457386.csv, 1
Processing ../submissions/merge_submission_5__score=0.28896974822665383.csv, 1
Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 1
Processing ../submissions/merge_submission_39__score=0.29576528082337394.csv, 1
Processing ../submissions/merge_submission_38__score=0.2861704325134863.csv, 1
Processing ../submissions/merge_submission_33__score=0.2960919791783309.csv, 1
Processing ../submissions/merge_submission_40__score=0.29589944253546246.csv, 1
Processing ../submissions/merge_submission_37__score=0.29609181013568253.csv, 1
Processing ../submissions/merge_submission_16__score=0.28969047017154304.csv, 1
Processing ../submissions/merge_submission_18__score=0.2952556100056696.csv, 1
Processing ../submissions/merge_submission_10__score=0.2900969049214374.csv, 1
Processing ../submissions/merge_submission_15__

,track,prediction
0,17730,"0.16021048463664883,0.20354254201546396,0.3504..."
1,32460,"0.2550976752785066,0.6006816619509386,0.550387..."
2,11288,"0.31617679785638286,0.3861452659275346,0.31135..."
3,18523,"0.5976513842522111,0.15643155262368857,0.44334..."
4,71342,"0.0902406361502388,0.01389829534834458,0.09305..."
...,...,...
25575,40192,"0.371003279211712,0.8279090502963082,0.7709767..."
25576,4481,"0.9076045571552842,0.25915006003782254,0.17101..."
25577,5257,"0.14438327029639692,0.1313716335138752,0.10386..."
25578,73095,"0.9873982865285773,0.06100679900047601,0.05275..."


In [229]:
merge_subm(
    ["../submissions/merge_submission_12__score=0.2833985712331001.csv", "../submissions/merge_submission_50__score=0.29610775216585283.csv"],
    scale=True, 
    weights=[0.2, 0.8],
    outname="../submissions/merge_submission_53.csv"
)

Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.2
Processing ../submissions/merge_submission_50__score=0.29610775216585283.csv, 0.8


,track,prediction
0,17730,"0.13233858890375877,0.16454124754942287,0.2541..."
1,32460,"0.21093057594304965,0.5676729442706399,0.49369..."
2,11288,"0.26042685589956105,0.3355113185753661,0.23277..."
3,18523,"0.5837425440381951,0.11924866867108064,0.37399..."
4,71342,"0.07397439093077629,0.012169499945659246,0.068..."
...,...,...
25575,40192,"0.30419471661639474,0.8488305124793485,0.73816..."
25576,4481,"0.9358731948264081,0.19833999904645422,0.11886..."
25577,5257,"0.11350023872101117,0.09467255568330213,0.0718..."
25578,73095,"0.991806116281895,0.04789414748538029,0.038617..."


In [233]:
merge_subm(
    ["../submissions/merge_submission_12__score=0.2833985712331001.csv", "../submissions/merge_submission_50__score=0.29610775216585283.csv"],
    scale=True, 
    weights=[0.4, 0.6],
    outname="../submissions/merge_submission_54.csv"
)

Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.4
Processing ../submissions/merge_submission_50__score=0.29610775216585283.csv, 0.6


,track,prediction
0,17730,"0.11580821767118588,0.1446003509361793,0.21180..."
1,32460,"0.1914784067625394,0.5417890849692788,0.483652..."
2,11288,"0.23653324222160818,0.30508420575182915,0.2069..."
3,18523,"0.5610807165206835,0.09929862342093038,0.34664..."
4,71342,"0.059660192626830644,0.008671808089937005,0.05..."
...,...,...
25575,40192,"0.27223784541577534,0.8542952597342,0.72881397..."
25576,4481,"0.9471471584077333,0.16784799176245835,0.09308..."
25577,5257,"0.09527865920275025,0.0753257853006008,0.05643..."
25578,73095,"0.9936350076079316,0.036664123492736234,0.0291..."


In [235]:
merge_subm(
    ["../submissions/merge_submission_12__score=0.2833985712331001.csv", "../submissions/merge_submission_50__score=0.29610775216585283.csv"],
    rank=True, 
    weights=[0.4, 0.6],
    outname="../submissions/merge_submission_55.csv"
)

Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.4
Processing ../submissions/merge_submission_50__score=0.29610775216585283.csv, 0.6


,track,prediction
0,17730,"0.2664972634870993,0.36071149335418295,0.63403..."
1,32460,"0.3941282251759187,0.6763096168881939,0.886786..."
2,11288,"0.4463330727130571,0.5510711493354183,0.632079..."
3,18523,"0.651618451915559,0.25139953088350275,0.798905..."
4,71342,"0.116278342455043,0.0013917122752150116,0.1627..."
...,...,...
25575,40192,"0.4782251759186865,0.799546520719312,0.9696637..."
25576,4481,"0.8460203283815482,0.3945660672400313,0.306528..."
25577,5257,"0.2150117279124316,0.16871774824081315,0.17594..."
25578,73095,"0.9809773260359655,0.039757623143080525,0.0551..."


In [238]:
merge_subm(
    ["../submissions/merge_submission_12__score=0.2833985712331001.csv", "../submissions/merge_submission_50__score=0.29610775216585283.csv"],
    rank=True, 
    weights=[0.1, 0.9],
    outname="../submissions/merge_submission_56.csv"
)

Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.1
Processing ../submissions/merge_submission_50__score=0.29610775216585283.csv, 0.9


,track,prediction
0,17730,"0.24614933541829556,0.34980453479280693,0.6681..."
1,32460,"0.38401876465989054,0.684519155590305,0.875762..."
2,11288,"0.4464151681000782,0.5612627052384676,0.621477..."
3,18523,"0.6581157154026583,0.250086004691165,0.7994331..."
4,71342,"0.1116927286942924,0.0007584050039093043,0.156..."
...,...,...
25575,40192,"0.48965989053948394,0.7984206411258796,0.96767..."
25576,4481,"0.8360750586395622,0.41338936669272874,0.33350..."
25577,5257,"0.2063330727130571,0.18417513682564504,0.16765..."
25578,73095,"0.9752775605942142,0.04697028928850665,0.05792..."


In [246]:
merge_subm(
    ["../submissions/merge_submission_12__score=0.2833985712331001.csv", "../submissions/merge_submission_50__score=0.29610775216585283.csv"],
    rank=True, 
    weights=[0.05, 0.95],
    outname="../submissions/merge_submission_57.csv"
)

Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.05
Processing ../submissions/merge_submission_50__score=0.29610775216585283.csv, 0.95


,track,prediction
0,17730,"0.2427580140734949,0.34798670836591084,0.67379..."
1,32460,"0.38233385457388586,0.6858874120406567,0.87392..."
2,11288,"0.4464288506645817,0.5629612978889756,0.619710..."
3,18523,"0.6591985926505082,0.24986708365910867,0.79952..."
4,71342,"0.11092845973416732,0.0006528537920250196,0.15..."
...,...,...
25575,40192,"0.4915656763096169,0.7982329945269742,0.967337..."
25576,4481,"0.8344175136825646,0.41652658326817826,0.33799..."
25577,5257,"0.20488663017982797,0.18675136825645036,0.1662..."
25578,73095,"0.9743275996872557,0.04817240031274433,0.05837..."


In [248]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv",
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
],weights=[0.8, 0.2], outname="../submissions/merge_submission_58.csv", scale=True)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.8
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.2


,track,prediction
0,17730,"0.15243190820655278,0.17703072611190793,0.2960..."
1,32460,"0.23293329103825508,0.5601460417530058,0.51273..."
2,11288,"0.2851822220300425,0.3340671475677543,0.274915..."
3,18523,"0.5660343973821009,0.14293162801651915,0.38677..."
4,71342,"0.09054872217548432,0.029908183712431036,0.090..."
...,...,...
25575,40192,"0.3387245748512592,0.8328206538785456,0.742395..."
25576,4481,"0.935686950044905,0.21462402277458648,0.151944..."
25577,5257,"0.13707891480383094,0.12824817648229747,0.1034..."
25578,73095,"0.9944589026162787,0.08121279131165071,0.06119..."


In [249]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv",
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
],weights=[0.75, 0.25], outname="../submissions/merge_submission_59.csv", scale=True)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.75
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.25


,track,prediction
0,17730,"0.1473837373666569,0.17169277923793208,0.28314..."
1,32460,"0.22703034731612737,0.5544109280124202,0.50919..."
2,11288,"0.27803666288050755,0.3267988361726473,0.26602..."
3,18523,"0.561453659349836,0.13691050983491693,0.379282..."
4,71342,"0.08621690065792426,0.028111433836823796,0.085..."
...,...,...
25575,40192,"0.32918226767099323,0.8350326655344534,0.73980..."
25576,4481,"0.9384179411677891,0.20656654221896026,0.14359..."
25577,5257,"0.13140646425791275,0.12183678376254381,0.0977..."
25578,73095,"0.9947844667934486,0.07673695607700287,0.05748..."


In [251]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv",
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
],weights=[0.55, 0.45], outname="../submissions/merge_submission_60.csv", scale=True)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.55
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.45


,track,prediction
0,17730,"0.12636026063347522,0.14939576041579783,0.2313..."
1,32460,"0.20244365115614477,0.5304465420662645,0.49497..."
2,11288,"0.24827407342956273,0.29643643018996163,0.2303..."
3,18523,"0.5423498187728659,0.11176111553585416,0.34922..."
4,71342,"0.06817925613069281,0.020606941548107216,0.065..."
...,...,...
25575,40192,"0.28943943526348603,0.8442519070155019,0.72940..."
25576,4481,"0.9497270295441329,0.1729110100069029,0.110072..."
25577,5257,"0.10778528990018926,0.09505780747234899,0.0747..."
25578,73095,"0.9960768544017953,0.05804256560380988,0.04258..."


In [253]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv",
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
],weights=[0.87, 0.13], outname="../submissions/merge_submission_61.csv", scale=True)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.87
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.13


,track,prediction
0,17730,"0.15936796775085033,0.18435407735304432,0.3141..."
1,32460,"0.24104378698655776,0.5680142827306256,0.51767..."
2,11288,"0.29500004026764737,0.3440388461501862,0.28734..."
3,18523,"0.5723282160257807,0.15119225037738254,0.39723..."
4,71342,"0.0965005357994636,0.0323732195665139,0.097576..."
...,...,...
25575,40192,"0.35183546449652087,0.8297859031235921,0.74601..."
25576,4481,"0.9319346370499546,0.2256784153379816,0.163616..."
25577,5257,"0.14487271893534553,0.13704423270768468,0.1114..."
25578,73095,"0.9940115856395049,0.08735337575256835,0.06637..."


In [ ]:
merge_subm([
    "../submissions/merge_submission_7__score=0.2907024506739986.csv",
    "../submissions/submission__cfg=43_44_45_46__score_threshold=-2__score=0.28227115176735484.csv",
],weights=[0.9, 0.1], outname="../submissions/merge_submission_14")

In [254]:
merge([43, 44, 45, 46, 72, 73, 74, 75], fold_idx=range(-100, 100), score_threshold=-2)

Processing prediction__cfg=43__fold_idx=1__epoch=17__score=0.25667.csv
Processing prediction__cfg=43__fold_idx=-1__epoch=18__score=-1.csv
Processing prediction__cfg=43__fold_idx=1__epoch=16__score=0.25620.csv
Processing prediction__cfg=43__fold_idx=2__epoch=16__score=0.25976.csv
Processing prediction__cfg=43__fold_idx=0__epoch=16__score=0.25611.csv
Processing prediction__cfg=43__fold_idx=0__epoch=17__score=0.25746.csv
Processing prediction__cfg=43__fold_idx=-1__epoch=19__score=-1.csv
Processing prediction__cfg=43__fold_idx=2__epoch=18__score=0.26178.csv
Processing prediction__cfg=44__fold_idx=0__epoch=18__score=0.25767.csv
Processing prediction__cfg=44__fold_idx=2__epoch=18__score=0.25624.csv
Processing prediction__cfg=44__fold_idx=1__epoch=17__score=0.25373.csv
Processing prediction__cfg=44__fold_idx=0__epoch=17__score=0.25425.csv
Processing prediction__cfg=44__fold_idx=-1__epoch=18__score=-1.csv
Processing prediction__cfg=44__fold_idx=-1__epoch=19__score=-1.csv
Processing prediction_

,track,prediction
0,17730,"0.4585578089583333,0.4950108697916667,0.572418..."
1,32460,"0.5474335935416667,0.6774288575000001,0.654845..."
2,11288,"0.570735756875,0.6212951802083332,0.5612974377..."
3,18523,"0.7165495208333333,0.46275547270833356,0.60047..."
4,71342,"0.35353290583333336,0.26720943270833325,0.3708..."
...,...,...
25575,40192,"0.6056235712500002,0.8045920002083334,0.732825..."
25576,4481,"0.8811938066666666,0.5381842785416667,0.479594..."
25577,5257,"0.44120038833333347,0.46598708562499996,0.3920..."
25578,73095,"0.9887469172916669,0.3779654385416666,0.316919..."


In [255]:
merge_subm([
    "../submissions/merge_submission_7__score=0.2907024506739986.csv",
    "../submissions/submission__cfg=58_59_60_61_62_63_64_65_66_67_68_69_70_71__score_threshold=-2__score=0.2831911741187932.csv",
],weights=[0.9, 0.1], outname="../submissions/merge_submission_62.csv")

Processing ../submissions/merge_submission_7__score=0.2907024506739986.csv, 0.9
Processing ../submissions/submission__cfg=58_59_60_61_62_63_64_65_66_67_68_69_70_71__score_threshold=-2__score=0.2831911741187932.csv, 0.1


,track,prediction
0,17730,"0.22771907174059342,0.2598614850446767,0.36257..."
1,32460,"0.3066012057156028,0.6122772633231711,0.514798..."
2,11288,"0.35979748286903135,0.4109830364374807,0.33187..."
3,18523,"0.612000846349729,0.23148734835910428,0.420155..."
4,71342,"0.16934728375573618,0.11706307730376068,0.1696..."
...,...,...
25575,40192,"0.41661413269713643,0.8359931342013323,0.69545..."
25576,4481,"0.9293617081971557,0.305482869146143,0.2303657..."
25577,5257,"0.21700304430178938,0.21534721094285147,0.1805..."
25578,73095,"0.9905376551991717,0.17106169227402643,0.14461..."


In [256]:
merge_subm([
    "../submissions/merge_submission_62.csv",
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
],weights=[0.8, 0.2], outname="../submissions/merge_submission_63.csv", scale=True)

Processing ../submissions/merge_submission_62.csv, 0.8
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.2


,track,prediction
0,17730,"0.14478052248838186,0.1685750392965051,0.29026..."
1,32460,"0.22697084773434711,0.5554440937043008,0.51422..."
2,11288,"0.27990619797592803,0.3275572153712542,0.26865..."
3,18523,"0.5637825113405701,0.13456406971781576,0.38278..."
4,71342,"0.08382106780231552,0.024078341766456124,0.083..."
...,...,...
25575,40192,"0.3326217895585953,0.8324855874871193,0.741750..."
25576,4481,"0.9369360541625447,0.20799911049966985,0.14548..."
25577,5257,"0.13046629278868493,0.11603632873523878,0.0950..."
25578,73095,"0.9944548865573264,0.07366376871419261,0.05752..."


In [ ]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv",
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
],weights=[0.8, 0.2], outname="../submissions/merge_submission_58.csv", scale=True)

In [261]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv"]
    ,weights=[0.34, 0.33, 0.33],
    outname="../submissions/merge_submission_64.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.34
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.33
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.33


,track,prediction
0,17730,"0.12507786579604566,0.1573648114808734,0.21582..."
1,32460,"0.19270199960934686,0.5350623265529366,0.42910..."
2,11288,"0.24282560416858323,0.2914672392195228,0.20784..."
3,18523,"0.5522383423526379,0.12208280949254689,0.30485..."
4,71342,"0.08500101567087033,0.05973578278408788,0.0824..."
...,...,...
25575,40192,"0.28590313274261436,0.8247077637025946,0.62001..."
25576,4481,"0.8930482516568192,0.1783495065516282,0.109009..."
25577,5257,"0.11194654659115724,0.10916880857516968,0.0868..."
25578,73095,"0.9313271749396066,0.07833716001572995,0.06645..."


In [ ]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv"]
    ,weights=[0.34, 0.33, 0.33],
    outname="../submissions/merge_submission_64.csv"
)

In [293]:
submissions_to_merge = [
    os.path.join("../submissions/", x) 
    for x in os.listdir("../submissions/") 
    if x.startswith("merge_submission_") and float(x.rsplit("__score=", 1)[1].rsplit(".", 1)[0]) > 0.2945
]

In [294]:
submissions_to_merge

['../submissions/merge_submission_27__score=0.2954113579821891.csv',
 '../submissions/merge_submission_65__score=0.296205520698101.csv',
 '../submissions/merge_submission_39__score=0.29576528082337394.csv',
 '../submissions/merge_submission_33__score=0.2960919791783309.csv',
 '../submissions/merge_submission_40__score=0.29589944253546246.csv',
 '../submissions/merge_submission_37__score=0.29609181013568253.csv',
 '../submissions/merge_submission_55__score=0.29474208063157936.csv',
 '../submissions/merge_submission_56__score=0.29635290233805034.csv',
 '../submissions/merge_submission_18__score=0.2952556100056696.csv',
 '../submissions/merge_submission_44__score=0.29605518919305496.csv',
 '../submissions/merge_submission_50__score=0.29610775216585283.csv',
 '../submissions/merge_submission_23__score=0.2946765556763446.csv',
 '../submissions/merge_submission_35__score=0.29607369180333576.csv',
 '../submissions/merge_submission_43__score=0.29597936723899243.csv',
 '../submissions/merge_sub

In [275]:
merge_subm(
    submissions_to_merge,
    outname="../submissions/merge_submission_65.csv"
)

Processing ../submissions/merge_submission_27__score=0.2954113579821891.csv, 1
Processing ../submissions/merge_submission_39__score=0.29576528082337394.csv, 1
Processing ../submissions/merge_submission_33__score=0.2960919791783309.csv, 1
Processing ../submissions/merge_submission_40__score=0.29589944253546246.csv, 1
Processing ../submissions/merge_submission_37__score=0.29609181013568253.csv, 1
Processing ../submissions/merge_submission_55__score=0.29474208063157936.csv, 1
Processing ../submissions/merge_submission_56__score=0.29635290233805034.csv, 1
Processing ../submissions/merge_submission_18__score=0.2952556100056696.csv, 1
Processing ../submissions/merge_submission_44__score=0.29605518919305496.csv, 1
Processing ../submissions/merge_submission_50__score=0.29610775216585283.csv, 1
Processing ../submissions/merge_submission_23__score=0.2946765556763446.csv, 1
Processing ../submissions/merge_submission_35__score=0.29607369180333576.csv, 1
Processing ../submissions/merge_submission_4

,track,prediction
0,17730,"0.16330508349807527,0.209013620648864,0.365359..."
1,32460,"0.25486122638092684,0.598694556725494,0.573674..."
2,11288,"0.3108794816258242,0.38924343522758387,0.33058..."
3,18523,"0.6009614643705009,0.1596208508844758,0.468648..."
4,71342,"0.08875580752409362,0.019137178057158703,0.097..."
...,...,...
25575,40192,"0.3620573616948994,0.8301164758269752,0.783440..."
25576,4481,"0.9110126440173791,0.25686561795006096,0.18216..."
25577,5257,"0.14291362418419898,0.13205259918312434,0.1081..."
25578,73095,"0.9877217369152876,0.06494267287047804,0.05465..."


In [361]:
def make_neural_predict(path, outpath=None):
    import torch
    from torch import nn
    from torch.utils.data import Dataset, DataLoader
    
    def get_test_features(path):
        df = pd.read_csv(path)
        av = np.vstack(
            df["prediction"]
            .str.split(",").apply(lambda x: list(map(float, x)))
        )
        # X_test = StandardScaler().fit_transform(X_test)
        X_test = av.argsort(axis=0).argsort(axis=0) / len(av)
        labels = df["track"].values
        return X_test, labels
    
    class SecondLevelDatasetSimpleTest(Dataset):
        def __init__(self, X_test):
            self.X = torch.from_numpy(X_test).float()

        def __getitem__(self, idx):
            return self.X[idx]

        def __len__(self):
            return len(self.X)
        
        
    @torch.no_grad()
    def predict(model, loader):
        model.eval()
        predictions = []
        for data in loader:
            x = data
            preds = torch.sigmoid(model(x)).numpy()
            predictions.append(preds)
        predictions = np.vstack(predictions)
        return predictions
    
    model = nn.Sequential(
        nn.Linear(256, 256),
        nn.ReLU(),
        nn.Linear(256, 256),
    )
    model.load_state_dict(torch.load("model_simple.pt"))
    
    X_test, labels_test = get_test_features(path)
    test_dataset = SecondLevelDatasetSimpleTest(X_test)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    predictions = predict(model, test_loader)
    predictions_df = pd.DataFrame(
        [
            {"track": track, "prediction": ",".join([str(p) for p in probs])}
            for track, probs in zip(labels_test, predictions)
        ]
    )
    base_path, name = path.rsplit("/", 1)
    name = name.split("__")[0]
    new_name = "nn_" + name
    if outpath is not None:
        base_path = outpath
    new_path = os.path.join(base_path, new_name)
    predictions_df.to_csv(new_path, index=False)
    return predictions_df

In [297]:
for x in tqdm(submissions_to_merge):
    make_neural_predict(x)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [06:51<00:00, 12.10s/it]


In [298]:
submissions_to_merge = [
    os.path.join("../submissions/", x) 
    for x in os.listdir("../submissions/") 
    if x.startswith("nn_merge_subm")
]

In [300]:
merge_subm(
    submissions_to_merge,
    outname="../submissions/merge_submission_66.csv"
)

Processing ../submissions/nn_merge_submission_65, 1
Processing ../submissions/nn_merge_submission_53, 1
Processing ../submissions/nn_merge_submission_55, 1
Processing ../submissions/nn_merge_submission_52, 1
Processing ../submissions/nn_merge_submission_64, 1
Processing ../submissions/nn_merge_submission_63, 1
Processing ../submissions/nn_merge_submission_37, 1
Processing ../submissions/nn_merge_submission_39, 1
Processing ../submissions/nn_merge_submission_23, 1
Processing ../submissions/nn_merge_submission_24, 1
Processing ../submissions/nn_merge_submission_48, 1
Processing ../submissions/nn_merge_submission_25, 1
Processing ../submissions/nn_merge_submission_49, 1
Processing ../submissions/nn_merge_submission_47, 1
Processing ../submissions/nn_merge_submission_40, 1
Processing ../submissions/nn_merge_submission_35, 1
Processing ../submissions/nn_merge_submission_32, 1
Processing ../submissions/nn_merge_submission_59, 1
Processing ../submissions/nn_merge_submission_61, 1
Processing .

,track,prediction
0,17730,"0.028370818705882355,0.07457974564705883,0.138..."
1,32460,"0.12204807902941178,0.5044405873529412,0.37941..."
2,11288,"0.11459536038235296,0.34778636558823534,0.1036..."
3,18523,"0.6492386185294118,0.02212491105882353,0.24587..."
4,71342,"0.0070625256117647075,0.00022518593617647058,0..."
...,...,...
25575,40192,"0.1847031279411764,0.8843016141176471,0.661924..."
25576,4481,"0.9694425364705881,0.07795073408823529,0.02301..."
25577,5257,"0.03438416561764707,0.020171592588235292,0.001..."
25578,73095,"0.999950161470588,0.00543098848235294,0.000782..."


In [305]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
    "../submissions/merge_submission_66__score=0.27037682600485047.csv"
]
    ,weights=[0.4, 0.33, 0.33, 0.04],
    outname="../submissions/merge_submission_67.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.33
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.33
Processing ../submissions/merge_submission_66__score=0.27037682600485047.csv, 0.04


,track,prediction
0,17730,"0.12849294962714156,0.16133510445640092,0.2217..."
1,32460,"0.19748247399522081,0.5388748712947912,0.43198..."
2,11288,"0.24557346212356435,0.3011105083397517,0.21169..."
3,18523,"0.5595802283370861,0.1258352524063379,0.309535..."
4,71342,"0.0881165962385869,0.06212836498138618,0.08570..."
...,...,...
25575,40192,"0.29036036608647914,0.8276800713705242,0.62532..."
25576,4481,"0.8977889443789631,0.1828505114883329,0.113643..."
25577,5257,"0.11613933944589525,0.11338839284570679,0.0902..."
25578,73095,"0.9370314149922688,0.08219750913480764,0.06954..."


In [308]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
]
    ,weights=[0.4, 0.33, 0.33],
    outname="../submissions/merge_submission_68.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.33
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.33
Processing ../submissions/merge_submission_66__score=0.27037682600485047.csv, 0


,track,prediction
0,17730,"0.13227114324681175,0.16460889158128175,0.2249..."
1,32460,"0.20032905493732683,0.5401742782359931,0.43397..."
2,11288,"0.2505160320005912,0.29934915523603534,0.21577..."
3,18523,"0.5561968928581303,0.12974885019303656,0.31193..."
4,71342,"0.09117524041318398,0.06446433400196014,0.0887..."
...,...,...
25575,40192,"0.29434743167686794,0.8255434093800668,0.62394..."
25576,4481,"0.8950850352434301,0.1868089936543743,0.117063..."
25577,5257,"0.11922444034507443,0.11690600794976233,0.0936..."
25578,73095,"0.9346571226723323,0.08509435897074934,0.07214..."


In [309]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
]
    ,weights=[0.5, 0.25, 0.25],
    outname="../submissions/merge_submission_69.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.5
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.25
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.25


,track,prediction
0,17730,"0.15588543790841755,0.18838996261797591,0.2548..."
1,32460,"0.22536736788271566,0.5569559378015827,0.44995..."
2,11288,"0.275762385994557,0.32522413205792017,0.241806..."
3,18523,"0.5691921344165655,0.15491514542191678,0.33519..."
4,71342,"0.1114441600217894,0.07998735567679331,0.10942..."
...,...,...
25575,40192,"0.3220686150468924,0.8282866906444958,0.636869..."
25576,4481,"0.9017714460075569,0.21458003717349033,0.14350..."
25577,5257,"0.14311651579985307,0.14230590488655637,0.1157..."
25578,73095,"0.945588769269664,0.10727708281298468,0.090811..."


In [310]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_submission_11__score=0.2881629752287618.csv",
]
    ,weights=[0.4, 0.33, 0.33],
    outname="../submissions/merge_submission_70.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.33
Processing ../submissions/merge_submission_11__score=0.2881629752287618.csv, 0.33


,track,prediction
0,17730,"0.14655618440705612,0.17697877541772936,0.2380..."
1,32460,"0.21343035314019848,0.5586256152196475,0.44864..."
2,11288,"0.26917606400822464,0.3096683657176498,0.23318..."
3,18523,"0.5516958523262917,0.14298786023987617,0.33487..."
4,71342,"0.10110164230696962,0.07036783166374702,0.0983..."
...,...,...
25575,40192,"0.3115359240349914,0.8482937986323656,0.661156..."
25576,4481,"0.9440131250154383,0.20469283134783142,0.13108..."
25577,5257,"0.13270061660575844,0.1305037056950686,0.10484..."
25578,73095,"0.9878081976899656,0.09569772251713433,0.08015..."


In [311]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_submission_5__score=0.28896974822665383.csv",
]
    ,weights=[0.4, 0.33, 0.33],
    outname="../submissions/merge_submission_71.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.33
Processing ../submissions/merge_submission_5__score=0.28896974822665383.csv, 0.33


,track,prediction
0,17730,"0.14954453283583968,0.17944002224798278,0.2420..."
1,32460,"0.21728781617190795,0.5598495486075787,0.45432..."
2,11288,"0.26977419035284844,0.3155772062914226,0.23418..."
3,18523,"0.5571639562978441,0.14533917963112983,0.33618..."
4,71342,"0.10247415488183582,0.07008317312133887,0.0994..."
...,...,...
25575,40192,"0.3140682061278269,0.8510038738166812,0.663977..."
25576,4481,"0.9484149171017854,0.20554012692964646,0.13261..."
25577,5257,"0.1352860273242571,0.13128170506722542,0.10651..."
25578,73095,"0.9934266836740161,0.0969475512683222,0.081043..."


In [313]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_submission_10__score=0.2900969049214374.csv",
]
    ,weights=[0.4, 0.33, 0.33],
    outname="../submissions/merge_submission_72.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.33
Processing ../submissions/merge_submission_10__score=0.2900969049214374.csv, 0.33


,track,prediction
0,17730,"0.1721104434719623,0.20206453303001487,0.26458..."
1,32460,"0.2420117116915739,0.5668921785919776,0.471152..."
2,11288,"0.29036911272445504,0.3391958110019004,0.25679..."
3,18523,"0.5684987771664943,0.1677574258546238,0.355061..."
4,71342,"0.12156118686018516,0.08505273230995282,0.1182..."
...,...,...
25575,40192,"0.3335652368521996,0.8505319821241092,0.670999..."
25576,4481,"0.947689573503333,0.22828371682532328,0.156226..."
25577,5257,"0.15682080325125874,0.15396453741263078,0.1267..."
25578,73095,"0.9932727074372982,0.11850741282983525,0.09941..."


In [314]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
]
    ,weights=[0.42, 0.32, 0.32],
    outname="../submissions/merge_submission_73.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.42
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.32
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.32


,track,prediction
0,17730,"0.13590411165628954,0.1682675178946193,0.22952..."
1,32460,"0.20418110308277124,0.5427560720153145,0.43642..."
2,11288,"0.2544000864612014,0.30332992090094074,0.21978..."
3,18523,"0.5581961607901974,0.1336205879205566,0.315515..."
4,71342,"0.09429353573758481,0.0668524911827037,0.09193..."
...,...,...
25575,40192,"0.2986122291184101,0.8259654526515174,0.625934..."
25576,4481,"0.8961137138225265,0.19108146188808448,0.12113..."
25577,5257,"0.12290014426119421,0.12081368440157678,0.0970..."
25578,73095,"0.936338914456537,0.08850708571570859,0.075013..."


In [315]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
]
    ,weights=[0.4, 0.35, 0.35],
    outname="../submissions/merge_submission_74.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.35
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.35


,track,prediction
0,17730,"0.1296289844035551,0.16194807244430898,0.22158..."
1,32460,"0.19752756537700358,0.5382966100328502,0.43218..."
2,11288,"0.24769126512014747,0.296454052934286,0.212863..."
3,18523,"0.5547428798166272,0.12693304093665836,0.30933..."
4,71342,"0.08890738926816519,0.06272749241596481,0.0864..."
...,...,...
25575,40192,"0.29124576081029174,0.8252364688190117,0.62250..."
25576,4481,"0.8943369053677235,0.1837017440298578,0.114105..."
25577,5257,"0.11655120113335092,0.11406406143935179,0.0911..."
25578,73095,"0.9334340013747284,0.0826123758835062,0.070052..."


In [316]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
]
    ,weights=[0.4, 0.37, 0.35],
    outname="../submissions/merge_submission_75.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.37
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.35


,track,prediction
0,17730,"0.12840336437323954,0.16053165987440007,0.2190..."
1,32460,"0.19624894459821257,0.5368348998484401,0.43201..."
2,11288,"0.2460467000418465,0.2947959480625603,0.211237..."
3,18523,"0.5534564586171705,0.12531365604897643,0.30821..."
4,71342,"0.08753681744436391,0.06162361465586092,0.0850..."
...,...,...
25575,40192,"0.28900265114456175,0.8260832869703529,0.62299..."
25576,4481,"0.8959488266216611,0.1816784757441356,0.112350..."
25577,5257,"0.1150830607255075,0.11228652007740958,0.08970..."
25578,73095,"0.9346174666810623,0.08119172335984065,0.06882..."


In [317]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
]
    ,weights=[0.4, 0.35, 0.37],
    outname="../submissions/merge_submission_76.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.35
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.37


,track,prediction
0,17730,"0.12835398981436,0.1608462097595829,0.22093073..."
1,32460,"0.19615477639334586,0.537981241382143,0.430658..."
2,11288,"0.24666239011517135,0.2953721502704275,0.21173..."
3,18523,"0.5546531815303752,0.12588746349241087,0.30799..."
4,71342,"0.08813160197257368,0.06218757367503741,0.0856..."
...,...,...
25575,40192,"0.2905533605487264,0.8240991533509578,0.620641..."
25576,4481,"0.8920169326242777,0.18278422249237697,0.11306..."
25577,5257,"0.11548931157295608,0.11315190342536975,0.0902..."
25578,73095,"0.9310929391260201,0.08168400869960236,0.06930..."


In [318]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
]
    ,weights=[0.4, 0.4, 0.3],
    outname="../submissions/merge_submission_77.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.4
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.3


,track,prediction
0,17730,"0.12975466509888486,0.1611474000092982,0.21686..."
1,32460,"0.19776726626211885,0.5353786497652429,0.43562..."
2,11288,"0.24612405402441162,0.2949873564051696,0.21160..."
3,18523,"0.5516966760375606,0.1254724401715524,0.309901..."
4,71342,"0.08739339228726763,0.06129196036715192,0.0849..."
...,...,...
25575,40192,"0.2872985005087816,0.8302869907592901,0.628482..."
25576,4481,"0.9043453628156085,0.1808871159434252,0.112297..."
25577,5257,"0.11551710806711812,0.11186126746272591,0.0897..."
25578,73095,"0.94240552606029,0.08135928592774909,0.0688288..."


In [320]:
merge_subm([
    "../submissions/merge_submission_74__score=0.2970604529353247.csv", 
    "../submissions/merge_submission_58__score=0.29668032738560357.csv",
]
    ,weights=[0.5, 0.5],
    outname="../submissions/merge_submission_78.csv"
)

Processing ../submissions/merge_submission_74__score=0.2970604529353247.csv, 0.5
Processing ../submissions/merge_submission_58__score=0.29668032738560357.csv, 0.5


,track,prediction
0,17730,"0.14103044630505396,0.16948939927810847,0.2588..."
1,32460,"0.21523042820762933,0.5492213258929279,0.47245..."
2,11288,"0.266436743575095,0.31526060025102015,0.243889..."
3,18523,"0.560388638599364,0.13493233447658876,0.348053..."
4,71342,"0.08972805572182475,0.046317838064197925,0.088..."
...,...,...
25575,40192,"0.31498516783077546,0.8290285613487787,0.68244..."
25576,4481,"0.9150119277063142,0.19916288340222216,0.13302..."
25577,5257,"0.12681505796859094,0.12115611896082462,0.0972..."
25578,73095,"0.9639464519955036,0.08191258359757846,0.06562..."


In [321]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
],
    weights=[0.4, 0.35, 0.35],
    scale=True,
    outname="../submissions/merge_submission_79.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.35
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.35


,track,prediction
0,17730,"0.09998790070590924,0.12892642373140706,0.2240..."
1,32460,"0.17714189578437395,0.5588503718547804,0.47774..."
2,11288,"0.23541104588462022,0.2842170305006607,0.21006..."
3,18523,"0.5796776971104861,0.08996724330324646,0.32844..."
4,71342,"0.05416755907957427,0.01287067714054535,0.0509..."
...,...,...
25575,40192,"0.28743863719668206,0.8748181723750741,0.71261..."
25576,4481,"0.9508146501668513,0.1571106998093794,0.087400..."
25577,5257,"0.08624592086900212,0.07599234881323169,0.0572..."
25578,73095,"0.9972954516830774,0.03795112800699674,0.02978..."


In [322]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
],
    weights=[0.4, 0.35, 0.35],
    rank=True,
    outname="../submissions/merge_submission_80.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.35
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.35


,track,prediction
0,17730,"0.24639633236193043,0.36373231928353117,0.6658..."
1,32460,"0.38747068021892095,0.6831882152249625,0.88626..."
2,11288,"0.4514446655768,0.555323761461369,0.6560736370..."
3,18523,"0.6448539341815338,0.2598621081811074,0.791168..."
4,71342,"0.11696637998436277,0.012792309332575165,0.185..."
...,...,...
25575,40192,"0.4936296112019333,0.8160334778591227,0.960347..."
25576,4481,"0.8274895159570688,0.41137607505863955,0.32377..."
25577,5257,"0.2100326959982941,0.19876323832539625,0.20329..."
25578,73095,"0.9462381832397467,0.03355782216220058,0.05979..."


In [323]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
]
    ,weights=[0.4, 0.37, 0.33],
    outname="../submissions/merge_submission_81.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.37
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.33


,track,prediction
0,17730,"0.12967925668168703,0.16162780347030464,0.2196..."
1,32460,"0.19762344573104967,0.5371294259258074,0.43355..."
2,11288,"0.2470643806818531,0.2958673743226394,0.212359..."
3,18523,"0.5535243983050004,0.126348800630616,0.3095620..."
4,71342,"0.08830179047580616,0.06215327959643965,0.0858..."
...,...,...
25575,40192,"0.28966685668968767,0.8272566775951232,0.62489..."
25576,4481,"0.8983402883468775,0.18257589279528474,0.11338..."
25577,5257,"0.11613756390685781,0.11318294384870144,0.0905..."
25578,73095,"0.937022611248953,0.08211113990120336,0.069562..."


In [324]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
    "../submissions/submission__cfg=58_59_60_61_62_63_64_65_66_67_68_69_70_71__score_threshold=-2__score=0.2831911741187932.csv"
]
    ,weights=[0.37, 0.35, 0.35, 0.03],
    outname="../submissions/merge_submission_82.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.37
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.35
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.35
Processing ../submissions/submission__cfg=58_59_60_61_62_63_64_65_66_67_68_69_70_71__score_threshold=-2__score=0.2831911741187932.csv, 0.03


,track,prediction
0,17730,"0.12868084057368392,0.16082440551131713,0.2230..."
1,32460,"0.19801170025788822,0.5361500777263847,0.43595..."
2,11288,"0.2477978819406885,0.2962988308064754,0.214353..."
3,18523,"0.5545562901947197,0.12537241790321385,0.31142..."
4,71342,"0.08657023970977436,0.059030285458859286,0.084..."
...,...,...
25575,40192,"0.290801196061887,0.8233189578231078,0.6252618..."
25576,4481,"0.8931873868983642,0.18314020010739948,0.11474..."
25577,5257,"0.11568804589368523,0.11190281042155573,0.0894..."
25578,73095,"0.9334832410887202,0.08014777440385076,0.06812..."


In [ ]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
    "../submissions/submission__cfg=58_59_60_61_62_63_64_65_66_67_68_69_70_71__score_threshold=-2__score=0.2831911741187932.csv"
]
    ,weights=[0.37, 0.35, 0.35, 0.03],
    outname="../submissions/merge_submission_82.csv"
)

In [325]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/prediction_epoch_15_nan_stuck.csv",
]
    ,weights=[0.4, 0.35, 0.35],
    outname="../submissions/merge_submission_83.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.35
Processing ../submissions/prediction_epoch_15_nan_stuck.csv, 0.35


,track,prediction
0,17730,"0.11870657293262796,0.1379548876853297,0.20294..."
1,32460,"0.17934697564767665,0.5564279652986358,0.38995..."
2,11288,"0.2475766583069482,0.2543464696930374,0.194302..."
3,18523,"0.5349861432027329,0.11216471884976711,0.30404..."
4,71342,"0.07658055397019775,0.054464914531046,0.077668..."
...,...,...
25575,40192,"0.2830128647585442,0.8272063332164689,0.691739..."
25576,4481,"0.9608677334521157,0.18712271946092537,0.10912..."
25577,5257,"0.10977488843977472,0.11299715145778347,0.0821..."
25578,73095,"0.9947653873688465,0.07550584237922242,0.06199..."


In [380]:
submissions_to_predict = []
for x in os.listdir("../predictions_test"):
    try:
        if float(x.rsplit("__score=", 1)[1].rsplit(".", 1)[0]) > 0.26:
            submissions_to_predict.append(os.path.join("../predictions_test", x))
    except Exception:
        pass

In [382]:
# for x in tqdm(submissions_to_predict):
#     make_neural_predict(x)

In [383]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
    "../submissions/submission_0__score=0.27772388381897406.csv",
    "../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv"
]
    ,weights=[0.42, 0.35, 0.35, 0.1, 0.1],
    outname="../submissions/merge_submission_101.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.42
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.35
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.35
Processing ../submissions/submission_0__score=0.27772388381897406.csv, 0.1
Processing ../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv, 0.1


,track,prediction
0,17730,"0.1157028921263441,0.14434182101874934,0.21144..."
1,32460,"0.17981591893671114,0.5465928373938538,0.40607..."
2,11288,"0.23721683610765248,0.2853260687472395,0.19418..."
3,18523,"0.5560857659689847,0.11423241924672232,0.30264..."
4,71342,"0.0782017074558309,0.05504371216570923,0.07670..."
...,...,...
25575,40192,"0.27936178915659665,0.8292543152436713,0.62979..."
25576,4481,"0.9067061019765938,0.17898351108786806,0.10461..."
25577,5257,"0.10537473323667167,0.10457980215829628,0.0803..."
25578,73095,"0.9439088241538317,0.07320268008452965,0.06126..."


In [384]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
    "../submissions/submission_0__score=0.27772388381897406.csv",
    "../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv"
]
    ,weights=[0.40, 0.37, 0.35, 0.1, 0.1],
    outname="../submissions/merge_submission_102.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.37
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.35
Processing ../submissions/submission_0__score=0.27772388381897406.csv, 0.1
Processing ../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv, 0.1


,track,prediction
0,17730,"0.11282096030184831,0.1412743277861737,0.20694..."
1,32460,"0.17673775222208138,0.5440690931499681,0.40464..."
2,11288,"0.2337841315093731,0.281983669304717,0.1907657..."
3,18523,"0.5539642285916808,0.11088138053834921,0.29986..."
4,71342,"0.07544438026295452,0.052886712801846716,0.073..."
...,...,...
25575,40192,"0.2752505455610678,0.8297680603489409,0.629160..."
25576,4481,"0.9075516333435305,0.1750784771411848,0.101048..."
25577,5257,"0.10224589605303708,0.10107361155362642,0.0773..."
25578,73095,"0.944053431506954,0.07025254427984387,0.058758..."


In [385]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
    "../submissions/submission_0__score=0.27772388381897406.csv",
    "../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv"
]
    ,weights=[0.40, 0.35, 0.37, 0.1, 0.1],
    outname="../submissions/merge_submission_103.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.35
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.37
Processing ../submissions/submission_0__score=0.27772388381897406.csv, 0.1
Processing ../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv, 0.1


,track,prediction
0,17730,"0.11276456162102669,0.1415372786275104,0.20853..."
1,32460,"0.17668265739966837,0.5450059054641514,0.40352..."
2,11288,"0.23435902452645185,0.28237384271057736,0.1911..."
3,18523,"0.5549879938655026,0.11136672998698907,0.29967..."
4,71342,"0.07594995464668891,0.05336520492264155,0.0743..."
...,...,...
25575,40192,"0.2765949042916705,0.8280736480507935,0.627206..."
25576,4481,"0.9042119656641872,0.1760216943085366,0.101650..."
25577,5257,"0.10259538045143247,0.10180895188248898,0.0778..."
25578,73095,"0.9410630402734675,0.07067100541832491,0.05916..."


In [386]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
    "../submissions/submission_0__score=0.27772388381897406.csv",
    "../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv"
]
    ,weights=[0.40, 0.35, 0.35, 0.12, 0.1],
    outname="../submissions/merge_submission_104.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.35
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.35
Processing ../submissions/submission_0__score=0.27772388381897406.csv, 0.12
Processing ../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv, 0.1


,track,prediction
0,17730,"0.11224444678907775,0.14039474601993995,0.2076..."
1,32460,"0.1758169150316052,0.5458693033339508,0.401518..."
2,11288,"0.23435356705915664,0.28036871969908933,0.1902..."
3,18523,"0.5540471968838886,0.11066347655427997,0.29942..."
4,71342,"0.07536296248964285,0.05297174883288351,0.0739..."
...,...,...
25575,40192,"0.2762028616225397,0.8281674511173391,0.630503..."
25576,4481,"0.9073801003348724,0.17618459790049218,0.10141..."
25577,5257,"0.10227269889459553,0.10175814664527146,0.0774..."
25578,73095,"0.9439835824636638,0.07033259906097807,0.05878..."


In [387]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
    "../submissions/submission_0__score=0.27772388381897406.csv",
    "../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv"
]
    ,weights=[0.40, 0.35, 0.35, 0.1, 0.12],
    outname="../submissions/merge_submission_105.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.35
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.35
Processing ../submissions/submission_0__score=0.27772388381897406.csv, 0.1
Processing ../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv, 0.12


,track,prediction
0,17730,"0.1122410138913097,0.14064067292037555,0.20808..."
1,32460,"0.1760287767626146,0.5459476109493657,0.402760..."
2,11288,"0.23380231316571384,0.28323693628640145,0.1898..."
3,18523,"0.5561745245073604,0.11090315463664209,0.30050..."
4,71342,"0.07539677669656707,0.05289398686263297,0.0738..."
...,...,...
25575,40192,"0.27544937468401326,0.8306321022577808,0.62846..."
25576,4481,"0.9073371556520572,0.17605301496050735,0.10134..."
25577,5257,"0.10212584520173512,0.10111519501272255,0.0773..."
25578,73095,"0.9440386982098955,0.07035676546222301,0.05877..."


In [388]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
    "../submissions/submission_0__score=0.27772388381897406.csv",
    "../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv"
]
    ,weights=[0.38, 0.35, 0.35, 0.1, 0.1],
    outname="../submissions/merge_submission_106.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.38
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.35
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.35
Processing ../submissions/submission_0__score=0.27772388381897406.csv, 0.1
Processing ../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv, 0.1


,track,prediction
0,17730,"0.11143863560655551,0.13993551667804086,0.2062..."
1,32460,"0.17520245878670235,0.5441309425326117,0.40267..."
2,11288,"0.23279580507349398,0.280782682702531,0.189382..."
3,18523,"0.5540205481987549,0.10975480124814498,0.29866..."
4,71342,"0.07458054249762525,0.05228653064068459,0.0730..."
...,...,...
25575,40192,"0.2744954154662478,0.8289350489282257,0.627932..."
25576,4481,"0.9060084355273121,0.17433298189165117,0.10003..."
25577,5257,"0.10115136022603559,0.1001648168863793,0.07644..."
25578,73095,"0.9424635920476675,0.06931169066359603,0.05796..."


In [389]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
    "../submissions/submission_0__score=0.27772388381897406.csv",
    "../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv"
]
    ,weights=[0.4, 0.33, 0.35, 0.1, 0.1],
    outname="../submissions/merge_submission_107.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.33
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.35
Processing ../submissions/submission_0__score=0.27772388381897406.csv, 0.1
Processing ../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv, 0.1


,track,prediction
0,17730,"0.1144106278005668,0.1430988690741345,0.210911..."
1,32460,"0.17837681821116427,0.5467335537841189,0.40414..."
2,11288,"0.23633578169046957,0.2842295321276323,0.19291..."
3,18523,"0.5562083836190994,0.11321055991615478,0.30153..."
4,71342,"0.07742403616527901,0.054510936234667816,0.075..."
...,...,...
25575,40192,"0.27873513542413686,0.8284052492884167,0.62859..."
25576,4481,"0.9051364813051587,0.17836004814916828,0.10371..."
25577,5257,"0.10437797357165877,0.1037805759474451,0.07949..."
25578,73095,"0.9423144657147605,0.07235401821217827,0.06055..."


In [390]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
    "../submissions/submission_0__score=0.27772388381897406.csv",
    "../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv"
]
    ,weights=[0.4, 0.35, 0.33, 0.1, 0.1],
    outname="../submissions/merge_submission_108.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.35
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.33
Processing ../submissions/submission_0__score=0.27772388381897406.csv, 0.1
Processing ../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv, 0.1


,track,prediction
0,17730,"0.11446878894016413,0.14282770101900605,0.2092..."
1,32460,"0.1784336347467777,0.5457674660851173,0.405299..."
2,11288,"0.23574292326660712,0.28382716580283884,0.1924..."
3,18523,"0.5551526256804707,0.11271004329724493,0.30172..."
4,71342,"0.07690266258205293,0.05401749123509815,0.0754..."
...,...,...
25575,40192,"0.27734876548320286,0.8301526119708811,0.63060..."
25576,4481,"0.9085805135994814,0.17738735544533676,0.10308..."
25577,5257,"0.1040175677858135,0.10302225623330559,0.07901..."
25578,73095,"0.9453983066742929,0.0719224801631197,0.060135..."


In [393]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
    "../submissions/submission_0__score=0.27772388381897406.csv",
    "../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv"
]
    ,weights=[0.4, 0.35, 0.30, 0.1, 0.1],
    outname="../submissions/merge_submission_109.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.35
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.3
Processing ../submissions/submission_0__score=0.27772388381897406.csv, 0.1
Processing ../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv, 0.1


,track,prediction
0,17730,"0.115818536976921,0.1438497155530706,0.2098589..."
1,32460,"0.17982040880568828,0.5463706220969222,0.40670..."
2,11288,"0.23683897106881008,0.28497819769190996,0.1935..."
3,18523,"0.5552830140779255,0.11377394743896758,0.30334..."
4,71342,"0.07765720726686122,0.054534101994563766,0.076..."
...,...,...
25575,40192,"0.27794582354689645,0.8317991513955905,0.63329..."
25576,4481,"0.9120404035642347,0.1784689590656825,0.104229..."
25577,5257,"0.10514394015464328,0.10398319327915236,0.0799..."
25578,73095,"0.9488318376637466,0.07291364816099717,0.06090..."


In [394]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
    "../submissions/submission_0__score=0.27772388381897406.csv",
    "../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv"
]
    ,weights=[0.4, 0.35, 0.25, 0.1, 0.1],
    outname="../submissions/merge_submission_110.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.35
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.25
Processing ../submissions/submission_0__score=0.27772388381897406.csv, 0.1
Processing ../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv, 0.1


,track,prediction
0,17730,"0.11821808904226654,0.1456666302802965,0.21089..."
1,32460,"0.1822857849104182,0.5474428994512421,0.409192..."
2,11288,"0.2387875004949487,0.2870244766058141,0.195342..."
3,18523,"0.5555148156734006,0.11566533257980786,0.30623..."
4,71342,"0.07899862003985376,0.05545252112250265,0.0776..."
...,...,...
25575,40192,"0.27900726010457405,0.8347263325950739,0.63808..."
25576,4481,"0.9181913190571291,0.1803918099462972,0.106255..."
25577,5257,"0.10714637992145179,0.10569152580510213,0.0815..."
25578,73095,"0.9549358927561088,0.07467572460166823,0.06226..."


In [395]:
merge_subm([
    "../submissions/merge_submission_14__score=0.2912586993297299.csv", 
    "../submissions/merge_submission_12__score=0.2833985712331001.csv",
    "../submissions/merge_clf_submission_4__score=0.28498048405976684.csv",
    "../submissions/submission_0__score=0.27772388381897406.csv",
    "../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv"
]
    ,weights=[0.4, 0.35, 0.28, 0.1, 0.1],
    outname="../submissions/merge_submission_111.csv"
)

Processing ../submissions/merge_submission_14__score=0.2912586993297299.csv, 0.4
Processing ../submissions/merge_submission_12__score=0.2833985712331001.csv, 0.35
Processing ../submissions/merge_clf_submission_4__score=0.28498048405976684.csv, 0.28
Processing ../submissions/submission_0__score=0.27772388381897406.csv, 0.1
Processing ../submissions/merge_logreg_submission_24__score=0.29598460268633153__score=0.27520439099167193.csv, 0.1


,track,prediction
0,17730,"0.11675494753900705,0.1445587554466222,0.21026..."
1,32460,"0.18078250679777802,0.546789071796169,0.407672..."
2,11288,"0.2375993727960837,0.2857767455607506,0.194226..."
3,18523,"0.5553734732371353,0.11451204895734429,0.30447..."
4,71342,"0.07818068542217539,0.05489250945912529,0.0767..."
...,...,...
25575,40192,"0.27836004269135606,0.832941466010023,0.635168..."
25576,4481,"0.9144407608297545,0.1792193398971419,0.105020..."
25577,5257,"0.10592538006364173,0.10464985963074251,0.0805..."
25578,73095,"0.951213907943693,0.07360128774760051,0.061436..."
